#### This notebook is not optimized to run on Kaggle workspace due to the limited disk space available. Download and run this locally. You will need 30GB disk space, and min 16GB RAM. Also, in the same directory as the notebook store all the competition datasets in a folder named 'data' and create another folder in data named 'outputs'.

I post this here for anyone interested in learning or exploring another approach to the M5 project. There are plenty of exploratory analysis approaches out there. It would help to review one of those before starting this notebook since having an understanding of the datasets and the requirements of this competition is essential to benefit from this notebook.

<b>This is one of the two complementary competitions that together comprise the M5 forecasting challenge. Can you estimate, as precisely as possible, the point forecasts of the unit sales of various products sold in the USA by Walmart?</b>

How much camping gear will one store sell each month in a year? To the uninitiated, calculating sales at this level may seem as difficult as predicting the weather. Both types of forecasting rely on science and historical data. While a wrong weather forecast may result in you carrying around an umbrella on a sunny day, inaccurate business forecasts could result in actual or opportunity losses.


In [1]:
import time
x = time.time()

### Preprocessing the datasets

In the challenge, you are predicting item sales at stores in various locations for two 28-day time periods. Information about the data is found in the M5 Participants Guide.

#### Files
<b>calendar.csv</b> - Contains information about the dates on which the products are sold.

<b>sales_train_validation.csv</b> - Contains the historical daily unit sales data per product and store [d_1 - d_1913]

<b>sample_submission.csv</b> - The correct format for submissions. Reference the Evaluation tab for more info.

<b>sell_prices.csv</b> - Contains information about the price of the products sold per store and date.

<b>sales_train_evaluation.csv</b> - Includes sales [d_1 - d_1941] (labels used for the Public leaderboard)

### Engineering Features

<b>28 day rolling averages</b>  We will add the 28 day average of quantity_sold for each item within each state, store, and overall as three new features

When using for analysis with ML, the multiple  28-day period averages can be used for training. If we wanted to use a24-week period to determine latest trend for each ite, we will group data by item, then shift the averages by 28 days.

### 1. Import the relevant libraries

In [2]:
import pandas as pd
pd.set_option('max_rows', 999)
pd.set_option('max_columns', None)

import numpy as np

### 2.Save file paths as variables

In [3]:
#import sample_submission.csv from
sample_submission_path = 'data/sample_submission.csv'

#import sell_price.csv from
sell_prices_path = 'data/sell_prices.csv'

#import sales_train_validation.csv from
#sales_train_validation_path = 'data/sales_train_validation.csv'

#import sales_train_evaluation.csv from
sales_train_validation_path = 'data/sales_train_evaluation.csv'

#import calendar.csv from
calendar_path = 'data/calendar.csv'

#############################################
#Export files to folder
export_path = 'data/outputs/'

#name of the final preprocessed file to be exported
final_preprocessed_file = 'sales_train_evaluation_preprocessed_final.csv'  #input('Enter the name of the final preprocessed file: ')

#name of the preprocessed validation file to be exported
validation_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_prediction.csv'

#name of the preprocessed evaluation_data to be exported
evaluation_file = 'submission_kaggle_preprocessed_to_be_used_4_private_LB_prediction.csv'

#name of the file to be exported and used for submission
submission_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_submission.csv'

### 2. Import and review the  files

While reviewing the data, I test the impact of downcasting the datatypes. An, while downcasting, I also add the datatypes to a 'schema' variable.

#### sample_submission.csv

In [4]:
sample_submission_df = pd.read_csv(sample_submission_path)
sample_submission_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<b>sample_submission</b> file doesn't contain the calendar, sales, and price information. Therefore, once the sales_train_validation data is preprocessed, I will extract that information and merge with the sample_submission data before making predictions.

Each row contains an id that is a concatenation of an item_id and a store_id, which is either validation (corresponding to the Public leaderboard), or evaluation (corresponding to the Private leaderboard). We are predicting 28 forecast days (F1-F28) of items sold for each row. 

For the validation rows, this corresponds to d_1914 - d_1941, and for the evaluation rows, this corresponds to d_1942 - d_1969. (Note: a month before the competition close, the ground truth for the validation rows will be provided.)

#### sell_price.csv

In [5]:
#Explore prices
price_df = pd.read_csv(sell_prices_path)
price_df

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26
...,...,...,...,...
6841116,WI_3,FOODS_3_827,11617,1.00
6841117,WI_3,FOODS_3_827,11618,1.00
6841118,WI_3,FOODS_3_827,11619,1.00
6841119,WI_3,FOODS_3_827,11620,1.00


In [6]:
#check datatypes
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int64  
 3   sell_price  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 208.8+ MB


In [7]:
#downcast datatypes to reduce memory usage. downcast to int16. 
price_df['wm_yr_wk'] = price_df['wm_yr_wk'].astype('int16')
price_df['sell_price'] = price_df['sell_price'].astype('float32',)
#check the new datatypes
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841121 entries, 0 to 6841120
Data columns (total 4 columns):
 #   Column      Dtype  
---  ------      -----  
 0   store_id    object 
 1   item_id     object 
 2   wm_yr_wk    int16  
 3   sell_price  float32
dtypes: float32(1), int16(1), object(2)
memory usage: 143.5+ MB


It looks like sell_price.csv has the weekly prices for each item. To merge it with the sales_train_validation.csv, later perform a merge on 'store_id', 'item_id', 'wm_yr_wk' columns.

#### sales_train_validation.csv

In [8]:
#import sales_train data
# we need to use the sales_train_validation data
sales_train_validation = pd.read_csv(sales_train_validation_path)
sales_train_validation

id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
0         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
1         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
2         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
3         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
4         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
...        ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
30485     WI_3       WI    0    0    2    2    0    3    1    4    1     0   
30486     WI_3       WI    0    0    0    0    0    5    0    1    1     3   
30487     WI_3       WI    0    6    0    2    2    4    1    8    5     2   
30488     WI_3       WI    0    0    0    0    0    0    0    0    0     0   
30489     WI_3       WI    0    0    0    0    0    0    0    0    0     0   

       d_11  d_12  d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     0     3     4     4     0     0     1     0     1     1     7     7   
30486     1     1     0     4     2     0     1     2     1     1     0     0   
30487     7     5     3     5    20     8    10     3     3     4     7     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_23  d_24  d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     3     6     3     3     7    12     4     2     7     5    12     5   
30486     0     0     3     1     1     1     2     0     1     0     1     1   
30487     3     5     6     3     4     1     2     5     1     2     2     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_35  d_36  d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2      

In [9]:
#Check the memory usage and schema
sales_train_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int64(1941), object(6)
memory usage: 452.9+ MB


In [10]:
#check the maximum number of quantity sold in a day
max(sales_train_validation.describe().loc['max'])

763.0

In [11]:
#max quantity sold is 763. Therefore, we can downcast
#int64 to int8
for col in sales_train_validation.select_dtypes('int64').columns:
    sales_train_validation[col] = sales_train_validation[col].astype('int16')

#check the new datatypes
sales_train_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int16(1941), object(6)
memory usage: 114.3+ MB


Memory usage went down significantly.

#### calendar.csv

In [12]:
#importing the calendar_df.csv
calendar_df = pd.read_csv(calendar_path, parse_dates=['date'])
calendar_df

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [13]:
#Check columns, memory usage, and datatypes
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1969 non-null   datetime64[ns]
 1   wm_yr_wk      1969 non-null   int64         
 2   weekday       1969 non-null   object        
 3   wday          1969 non-null   int64         
 4   month         1969 non-null   int64         
 5   year          1969 non-null   int64         
 6   d             1969 non-null   object        
 7   event_name_1  162 non-null    object        
 8   event_type_1  162 non-null    object        
 9   event_name_2  5 non-null      object        
 10  event_type_2  5 non-null      object        
 11  snap_CA       1969 non-null   int64         
 12  snap_TX       1969 non-null   int64         
 13  snap_WI       1969 non-null   int64         
dtypes: datetime64[ns](1), int64(7), object(6)
memory usage: 215.5+ KB


<b>Calendar.csv</b> file has the characteristics of each day. The information is stored as a text in categorical column. These must be encoded and then merged with the sales data.

## 3. Preprocess the competition files

###  Process the calendar.csv

In [14]:
#importing the calendar_df.csv
calendar_df = pd.read_csv(calendar_path)

#int64 to int16
for col in calendar_df.select_dtypes('int64').columns:
    calendar_df[col] = calendar_df[col].astype('int16')

# Identify the non-numerical columns
categorical_columns_calendar_df = calendar_df.select_dtypes('object').copy()

# Split the 'd' values and extract the day number
calendar_df['day'] = categorical_columns_calendar_df['d'].map(lambda x: x.split('d_')[1] ).astype('int16')

# Identify the unique values of all the event Names
unique_event_names = []
for col in ['event_name_1', 'event_name_2']:
    unique_event_names.extend(categorical_columns_calendar_df[col].unique())
    
# Make a list a of the unique values
unique_event_names = list(set(unique_event_names))

#Remove np.nan values
unique_event_names.remove(np.nan)

# Identify the unique values of all the event Types
unique_event_types = []
for col in ['event_type_1', 'event_type_2']:
    unique_event_types.extend(categorical_columns_calendar_df[col].unique())

# Make a list a of the unique values
unique_event_types = list(set(unique_event_types))

# Remove the np.nan values
unique_event_types.remove(np.nan)

#add the unique event types and names as headers and fill null values with 0
calendar_df = calendar_df.reindex(columns=list(calendar_df.columns)+unique_event_names+ unique_event_types).fillna(0)

# If there is an event on a day, add 1 to the cell under the column of that event
for col in ['event_name_1','event_type_1', 'event_name_2', 'event_type_2']:  #choose the first column
        for indx, event in enumerate(calendar_df[calendar_df[col]!=0][col]):  #choose non_zero the cells 
            calendar_df[event][indx] = 1  #find the column with the event name and assign 1 at the right index
            calendar_df[event] = calendar_df[event].astype('int8')
            
#drop event_name and event_type columns since they area already encoded and to reduce memory usage
calendar_df.drop(['date', 'weekday', 'event_name_1', 'event_type_1', 'event_name_2','event_type_2'], axis=1, inplace=True )

calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1969 entries, 0 to 1968
Data columns (total 43 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   wm_yr_wk             1969 non-null   int16 
 1   wday                 1969 non-null   int16 
 2   month                1969 non-null   int16 
 3   year                 1969 non-null   int16 
 4   d                    1969 non-null   object
 5   snap_CA              1969 non-null   int16 
 6   snap_TX              1969 non-null   int16 
 7   snap_WI              1969 non-null   int16 
 8   day                  1969 non-null   int16 
 9   EidAlAdha            1969 non-null   int8  
 10  OrthodoxEaster       1969 non-null   int8  
 11  LaborDay             1969 non-null   int8  
 12  Chanukah End         1969 non-null   int8  
 13  LentStart            1969 non-null   int8  
 14  Halloween            1969 non-null   int8  
 15  NBAFinalsEnd         1969 non-null   int8  
 16  Cinco 

C:\Users\galip\anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### 4. Process the sales data

In [15]:
#Let's remember how the sales data looked like
sales_train_validation

id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
0         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
1         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
2         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
3         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
4         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
...        ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
30485     WI_3       WI    0    0    2    2    0    3    1    4    1     0   
30486     WI_3       WI    0    0    0    0    0    5    0    1    1     3   
30487     WI_3       WI    0    6    0    2    2    4    1    8    5     2   
30488     WI_3       WI    0    0    0    0    0    0    0    0    0     0   
30489     WI_3       WI    0    0    0    0    0    0    0    0    0     0   

       d_11  d_12  d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     0     3     4     4     0     0     1     0     1     1     7     7   
30486     1     1     0     4     2     0     1     2     1     1     0     0   
30487     7     5     3     5    20     8    10     3     3     4     7     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_23  d_24  d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     3     6     3     3     7    12     4     2     7     5    12     5   
30486     0     0     3     1     1     1     2     0     1     0     1     1   
30487     3     5     6     3     4     1     2     5     1     2     2     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_35  d_36  d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2      

In [16]:
#Check the datatypes and columns in sales data
sales_train_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int16(1941), object(6)
memory usage: 114.3+ MB


In [17]:
#Encode dept_id, cat_id, state_id and store_id
sales_train_validation = pd.concat([sales_train_validation,pd.get_dummies(sales_train_validation['dept_id'],drop_first=True)], axis=1)
sales_train_validation = pd.concat([sales_train_validation,pd.get_dummies(sales_train_validation['cat_id'],drop_first=True)], axis=1)
sales_train_validation = pd.concat([sales_train_validation,pd.get_dummies(sales_train_validation['state_id'],drop_first=True)], axis=1)
sales_train_validation = pd.concat([sales_train_validation,pd.get_dummies(sales_train_validation['store_id'],drop_first=True)], axis=1)

In [18]:
#check the output so far
sales_train_validation

id        item_id    dept_id   cat_id  \
0      HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES   
1      HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES   
2      HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES   
3      HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES   
4      HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES   
...                              ...            ...        ...      ...   
30485    FOODS_3_823_WI_3_evaluation    FOODS_3_823    FOODS_3    FOODS   
30486    FOODS_3_824_WI_3_evaluation    FOODS_3_824    FOODS_3    FOODS   
30487    FOODS_3_825_WI_3_evaluation    FOODS_3_825    FOODS_3    FOODS   
30488    FOODS_3_826_WI_3_evaluation    FOODS_3_826    FOODS_3    FOODS   
30489    FOODS_3_827_WI_3_evaluation    FOODS_3_827    FOODS_3    FOODS   

      store_id state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
0         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
1         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
2         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
3         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
4         CA_1       CA    0    0    0    0    0    0    0    0    0     0   
...        ...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
30485     WI_3       WI    0    0    2    2    0    3    1    4    1     0   
30486     WI_3       WI    0    0    0    0    0    5    0    1    1     3   
30487     WI_3       WI    0    6    0    2    2    4    1    8    5     2   
30488     WI_3       WI    0    0    0    0    0    0    0    0    0     0   
30489     WI_3       WI    0    0    0    0    0    0    0    0    0     0   

       d_11  d_12  d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     0     3     4     4     0     0     1     0     1     1     7     7   
30486     1     1     0     4     2     0     1     2     1     1     0     0   
30487     7     5     3     5    20     8    10     3     3     4     7     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_23  d_24  d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     0     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
30485     3     6     3     3     7    12     4     2     7     5    12     5   
30486     0     0     3     1     1     1     2     0     1     0     1     1   
30487     3     5     6     3     4     1     2     5     1     2     2     2   
30488     0     0     0     0     0     0     0     0     0     0     0     0   
30489     0     0     0     0     0     0     0     0     0     0     0     0   

       d_35  d_36  d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2      

In [19]:
#check the schema
sales_train_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1966 entries, id to WI_3
dtypes: int16(1941), object(6), uint8(19)
memory usage: 114.8+ MB


In [20]:
#Save the non-daily-sales column names to a varaible
sales_cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id','FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']

In [21]:
#melt the sales data on ['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
#then merge with calendar on ['d'], and
#merge with price data on ['store_id','item_id','wm_yr_wk']
#and export the merged data into the output folder to free up memory

sales_train_validation_processing = \
sales_train_validation.melt(id_vars=sales_cols, value_name='quantity_sold', var_name='d')\
.merge(calendar_df, on=['d'], how='left').merge(price_df, on=['store_id','item_id','wm_yr_wk'], how='left')

#Review the processed file
sales_train_validation_processing

,id,item_id,dept_id,cat_id,store_id,state_id,FOODS_2,FOODS_3,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2,HOBBIES,HOUSEHOLD,TX,WI,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,d,quantity_sold,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,day,EidAlAdha,OrthodoxEaster,LaborDay,Chanukah End,LentStart,Halloween,NBAFinalsEnd,Cinco De Mayo,LentWeek2,ColumbusDay,Pesach End,OrthodoxChristmas,Eid al-Fitr,Easter,Father's day,NBAFinalsStart,MartinLutherKingDay,ValentinesDay,SuperBowl,Ramadan starts,MemorialDay,PresidentsDay,NewYear,Thanksgiving,Christmas,Purim End,Mother's day,StPatricksDay,IndependenceDay,VeteransDay,Cultural,National,Religious,Sporting,sell_price
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,1,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.98
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.48
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,2,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.98
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.28


## Feature Engineering

#### As a new feature, we can add average weekly or monthly sales to the sales_train_validation_processing df.

In [22]:
###Calculate average item sales per state
#Group the temporary dataframe by state_id and item_id and day to find the weekly  sale averages in each state per item,
weekly_state_avg = sales_train_validation_processing[['state_id', 'item_id','wm_yr_wk', 'quantity_sold']].groupby(by=['state_id', 'item_id','wm_yr_wk']).mean()

weekly_state_avg.head()

quantity_sold
state_id item_id     wm_yr_wk               
CA       FOODS_1_001 11101          1.214286
                     11102          1.071429
                     11103          1.500000
                     11104          1.714286
                     11105          1.428571

#### First, we will add 4 to the wm_yr_wk column prior to merging with the main df to align an item with its average sales data from 4 weeks ago.

In [23]:
#reset the index
weekly_state_avg = weekly_state_avg.reset_index()

#rename the columns
weekly_state_avg.columns = ['state_id', 'item_id','wm_yr_wk', 'weekly_avg_4weeks_ago']

#shift the 'wm_yr_wk' column by 4. And if the week is more than 48,we need to add 52 weeks to shift the year.
weekly_state_avg['wm_yr_wk'] = [x+4 if int(str(x)[-2:])<49 else x+52 for x in weekly_state_avg.wm_yr_wk]

#Review the output
weekly_state_avg

,state_id,item_id,wm_yr_wk,weekly_avg_4weeks_ago
0,CA,FOODS_1_001,11105,1.214286
1,CA,FOODS_1_001,11106,1.071429
2,CA,FOODS_1_001,11107,1.500000
3,CA,FOODS_1_001,11108,1.714286
4,CA,FOODS_1_001,11109,1.428571
...,...,...,...,...
2542861,WI,HOUSEHOLD_2_516,11617,0.142857
2542862,WI,HOUSEHOLD_2_516,11618,0.000000
2542863,WI,HOUSEHOLD_2_516,11619,0.095238
2542864,WI,HOUSEHOLD_2_516,11620,0.190476


#### Now, it is time to merge this data with the sales_train_validation_processing df

In [24]:
# The latest version of the sales_train_validation is sales_train_validation_processing df
# We will left merge the weekly average data on ['state_id,'item_id','wm_yr_wk']
sales_train_validation_processing = sales_train_validation_processing.merge(weekly_state_avg, how='left' )

#Review the output
sales_train_validation_processing

,id,item_id,dept_id,cat_id,store_id,state_id,FOODS_2,FOODS_3,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2,HOBBIES,HOUSEHOLD,TX,WI,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,d,quantity_sold,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,day,EidAlAdha,OrthodoxEaster,LaborDay,Chanukah End,LentStart,Halloween,NBAFinalsEnd,Cinco De Mayo,LentWeek2,ColumbusDay,Pesach End,OrthodoxChristmas,Eid al-Fitr,Easter,Father's day,NBAFinalsStart,MartinLutherKingDay,ValentinesDay,SuperBowl,Ramadan starts,MemorialDay,PresidentsDay,NewYear,Thanksgiving,Christmas,Purim End,Mother's day,StPatricksDay,IndependenceDay,VeteransDay,Cultural,National,Religious,Sporting,sell_price,weekly_avg_4weeks_ago
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59394515,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,1,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.98,0.619048
59394516,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.48,0.666667
59394517,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,2,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.98,0.714286
59394518,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.28,1.952381


In [25]:
#Free the memory by deleting variables
del(price_df, sales_train_validation, weekly_state_avg, calendar_df )

In [26]:
#Now, Create a varaible to store the monthly state avg
#Group the temporary dataframe by state_id and item_id and day to find the  monthly sale averages in each state per item,
monthly_state_avg = sales_train_validation_processing[['state_id', 'item_id','year','month', 'quantity_sold']].groupby(by=['state_id', 'item_id','year','month']).mean()

#Review the output
monthly_state_avg

quantity_sold
state_id item_id         year month               
CA       FOODS_1_001     2011 1           0.916667
                              2           1.437500
                              3           1.161290
                              4           1.116667
                              5           1.604839
...                                            ...
WI       HOUSEHOLD_2_516 2016 1           0.086022
                              2           0.091954
                              3           0.053763
                              4           0.055556
                              5           0.090909

[594555 rows x 1 columns]

#### Next, we will add 1 to the month column of the monthly_state_avg df prior to merging with the main df to align each item with its previous month average sales data

In [27]:
#Adding 1 to the month count to align each item with sales average from previous month
#monthly state average variable has a 4 level multiindex which is stored as a tuple.We need to modify the month at the 4th level
monthly_state_avg.index = pd.MultiIndex.from_tuples([(x[0], x[1],x[2], x[3]+1) if x[3]<12 else (x[0], x[1],x[2]+1, 1) for x in monthly_state_avg.index ])

#reset the index to unlevel the multiindex
monthly_state_avg = monthly_state_avg.reset_index()

#rename the columns after reindex step
monthly_state_avg.columns = ['state_id', 'item_id','year','month', 'last_month_avg']

#The months should be starting from 2011 February now.
monthly_state_avg

,state_id,item_id,year,month,last_month_avg
0,CA,FOODS_1_001,2011,2,0.916667
1,CA,FOODS_1_001,2011,3,1.437500
2,CA,FOODS_1_001,2011,4,1.161290
3,CA,FOODS_1_001,2011,5,1.116667
4,CA,FOODS_1_001,2011,6,1.604839
...,...,...,...,...,...
594550,WI,HOUSEHOLD_2_516,2016,2,0.086022
594551,WI,HOUSEHOLD_2_516,2016,3,0.091954
594552,WI,HOUSEHOLD_2_516,2016,4,0.053763
594553,WI,HOUSEHOLD_2_516,2016,5,0.055556


#### Now it is time to merge previous month and weekly average from 28 days ago with the sales_train_validation df

In [28]:
# The latest version of the sales_train_validation is sales_train_validation_processing df
# We will left merge the monthly average data on ['state_id,'year','month']
sales_train_validation_processing = sales_train_validation_processing.merge(monthly_state_avg, how='left' )

#Review the output
sales_train_validation_processing

,id,item_id,dept_id,cat_id,store_id,state_id,FOODS_2,FOODS_3,HOBBIES_1,HOBBIES_2,HOUSEHOLD_1,HOUSEHOLD_2,HOBBIES,HOUSEHOLD,TX,WI,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3,d,quantity_sold,wm_yr_wk,wday,month,year,snap_CA,snap_TX,snap_WI,day,EidAlAdha,OrthodoxEaster,LaborDay,Chanukah End,LentStart,Halloween,NBAFinalsEnd,Cinco De Mayo,LentWeek2,ColumbusDay,Pesach End,OrthodoxChristmas,Eid al-Fitr,Easter,Father's day,NBAFinalsStart,MartinLutherKingDay,ValentinesDay,SuperBowl,Ramadan starts,MemorialDay,PresidentsDay,NewYear,Thanksgiving,Christmas,Purim End,Mother's day,StPatricksDay,IndependenceDay,VeteransDay,Cultural,National,Religious,Sporting,sell_price,weekly_avg_4weeks_ago,last_month_avg
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN,NaN,NaN
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN,NaN,NaN
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN,NaN,NaN
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,d_1,0,11101,1,1,2011,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59394515,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,1,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.98,0.619048,0.333333
59394516,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.48,0.666667,0.522222
59394517,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,2,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.98,0.714286,1.377778
59394518,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,d_1941,0,11617,2,5,2016,0,0,0,1941,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.28,1.952381,1.388889


In [29]:
#save the datatypes as a variable
schema = dict(zip(sales_train_validation_processing.columns, sales_train_validation_processing.dtypes))
schema

{'id': dtype('O'),
 'item_id': dtype('O'),
 'dept_id': dtype('O'),
 'cat_id': dtype('O'),
 'store_id': dtype('O'),
 'state_id': dtype('O'),
 'FOODS_2': dtype('uint8'),
 'FOODS_3': dtype('uint8'),
 'HOBBIES_1': dtype('uint8'),
 'HOBBIES_2': dtype('uint8'),
 'HOUSEHOLD_1': dtype('uint8'),
 'HOUSEHOLD_2': dtype('uint8'),
 'HOBBIES': dtype('uint8'),
 'HOUSEHOLD': dtype('uint8'),
 'TX': dtype('uint8'),
 'WI': dtype('uint8'),
 'CA_2': dtype('uint8'),
 'CA_3': dtype('uint8'),
 'CA_4': dtype('uint8'),
 'TX_1': dtype('uint8'),
 'TX_2': dtype('uint8'),
 'TX_3': dtype('uint8'),
 'WI_1': dtype('uint8'),
 'WI_2': dtype('uint8'),
 'WI_3': dtype('uint8'),
 'd': dtype('O'),
 'quantity_sold': dtype('int16'),
 'wm_yr_wk': dtype('int16'),
 'wday': dtype('int16'),
 'month': dtype('int16'),
 'year': dtype('int16'),
 'snap_CA': dtype('int16'),
 'snap_TX': dtype('int16'),
 'snap_WI': dtype('int16'),
 'day': dtype('int16'),
 'EidAlAdha': dtype('int8'),
 'OrthodoxEaster': dtype('int8'),
 'LaborDay': dtype('int

In [30]:
#Reordering the columns 
organized_cols = [
         'id',
         'd', 'day', 

         'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'wm_yr_wk', 'year',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago', 

         'quantity_sold']

In [31]:
#Reorganize the columns in sales_train_validation_processing
sales_train_validation_processing = sales_train_validation_processing[organized_cols]

## Export the final preprocessed file

In [32]:
#Export the final preprocessed data
sales_train_validation_processing.to_csv(export_path+final_preprocessed_file)

In [33]:
#Free up memory
del(sales_train_validation_processing, monthly_state_avg, sample_submission_df)

###  Format the submission / validation file

#### Identify the training columns

In [1]:
import pandas as pd

#import sample_submission.csv from
sample_submission_path = 'data/sample_submission.csv'

#import sell_price.csv from
sell_prices_path = 'data/sell_prices.csv'

#import sales_train_validation.csv from
#sales_train_validation_path = 'data/sales_train_validation.csv'

#import sales_train_evaluation.csv from
sales_train_validation_path = 'data/sales_train_evaluation.csv'

#import calendar.csv from
calendar_path = 'data/calendar.csv'

#############################################
#Export files to folder
export_path = 'data/outputs/'

#name of the final preprocessed file to be exported
final_preprocessed_file = 'sales_train_evaluation_preprocessed_final.csv'  #input('Enter the name of the final preprocessed file: ')

#name of the preprocessed validation file to be exported
validation_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_prediction.csv'

#name of the preprocessed evaluation_data to be exported
evaluation_file = 'submission_kaggle_preprocessed_to_be_used_4_private_LB_prediction.csv'

#name of the file to be exported and used for submission
submission_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_submission.csv'

#Store the datatypes prior to export to be able to use later when importing
schema={'id': 'O', 'item_id': 'O', 'dept_id': 'O', 'cat_id': 'O', 'store_id': 'O',
        'state_id': 'O', 'FOODS_2': 'int8', 'FOODS_3': 'int8', 'HOBBIES_1': 'int8',
        'HOBBIES_2': 'int8', 'HOUSEHOLD_1': 'int8', 'HOUSEHOLD_2': 'int8',
        'HOBBIES': 'int8', 'HOUSEHOLD': 'int8', 'TX': 'int8', 'WI': 'int8',
        'CA_2': 'int8', 'CA_3': 'int8', 'CA_4': 'int8', 'TX_1': 'int8', 'TX_2': 'int8',
        'TX_3': 'int8', 'WI_1': 'int8', 'WI_2': 'int8', 'WI_3': 'int8', 'd': 'O',
        'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int8', 'month': 'int8',
        'year': 'int16', 'snap_CA': 'int8', 'snap_TX': 'int8', 'snap_WI': 'int8',
        'day': 'int16', 'EidAlAdha': 'int8', 'OrthodoxEaster': 'int8', 'LaborDay': 'int8',
        'Chanukah End': 'int8', 'LentStart': 'int8', 'Halloween': 'int8', 'NBAFinalsEnd': 'int8',
        'Cinco De Mayo': 'int8', 'LentWeek2': 'int8', 'ColumbusDay': 'int8', 'Pesach End': 'int8',
        'OrthodoxChristmas': 'int8', 'Eid al-Fitr': 'int8', 'Easter': 'int8', "Father's day": 'int8',
        'NBAFinalsStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
        'SuperBowl': 'int8', 'Ramadan starts': 'int8', 'MemorialDay': 'int8', 'PresidentsDay': 'int8',
        'NewYear': 'int8', 'Thanksgiving': 'int8', 'Christmas': 'int8', 'Purim End': 'int8',
        "Mother's day": 'int8', 'StPatricksDay': 'int8', 'IndependenceDay': 'int8', 'VeteransDay': 'int8',
        'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'Sporting': 'int8',
        'sell_price': 'float16', 'weekly_avg_4weeks_ago': 'float16', 'last_month_avg': 'float16'}


#identify the columns necessary when importing preprocessed file
training_cols = organized_cols = [
         'id', 'day', 'item_id', 'store_id','wm_yr_wk', 'year',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago', 

         'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]

#Reading the sample submission data to prepare for predictions
submission_file = pd.read_csv(sample_submission_path)

day_cols = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']

#set the first day in the training data
first_day = 1 # training_data.day_indx[0]

#set the last day in the training data
last_day = 1941 #training_data.day_indx[-1]

#set the last 28 days in the training data
next_28_days = list(range(first_day,last_day+1))[-28:]

#rename the col names of the submisson file to last 28 days
submission_file.rename(columns=dict(zip(list(day_cols), next_28_days)), inplace=True)


#add item_id
submission_file['item_id'] = submission_file.id.apply(lambda x: x.split('_')[0]+'_'+ x.split('_')[1]+ '_'+ x.split('_')[2])

# add store_id
submission_file['store_id'] = submission_file.id.apply(lambda x: x.split('_')[3]+'_'+x.split('_')[4])

#melt the file
submission_file = submission_file.melt(id_vars=['id','item_id','store_id'], var_name='day', value_name='quantity_sold')

#Review the submission file so far
submission_file

,id,item_id,store_id,day,quantity_sold
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,CA_1,1914,0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,CA_1,1914,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,CA_1,1914,0
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,CA_1,1914,0
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,CA_1,1914,0
...,...,...,...,...,...
1707435,FOODS_3_823_WI_3_evaluation,FOODS_3_823,WI_3,1941,0
1707436,FOODS_3_824_WI_3_evaluation,FOODS_3_824,WI_3,1941,0
1707437,FOODS_3_825_WI_3_evaluation,FOODS_3_825,WI_3,1941,0
1707438,FOODS_3_826_WI_3_evaluation,FOODS_3_826,WI_3,1941,0


In [2]:
#Now merge the submission file with the processed sales_train_evaluation file to extract sales, calendar, price data

#read the processed data in chunks to free up memory
sales_train_validation_processing = pd.read_csv(export_path+final_preprocessed_file, dtype=schema, index_col=0 , chunksize=100000)

validation_df = pd.DataFrame()

#read the sales_train_validation_process

for chunk in sales_train_validation_processing:
    
    #Extract the quantity sold
    temp_df = submission_file.merge(chunk, on=['item_id','store_id','day'], how='inner', suffixes=['','_y'])
    validation_df = pd.concat([validation_df, temp_df])
    
validation_df

,id,item_id,store_id,day,quantity_sold,id_y,d,dept_id,cat_id,state_id,...,Sporting,Cultural,National,Religious,wday,month,sell_price,last_month_avg,weekly_avg_4weeks_ago,quantity_sold_y
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,CA_1,1914,0,HOBBIES_1_001_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,CA,...,0,0,0,0,3,4,8.382812,0.991699,0.928711,0
1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,CA_1,1914,0,HOBBIES_1_001_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,CA,...,0,0,0,0,3,4,8.382812,0.991699,0.928711,0
2,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,CA_1,1914,0,HOBBIES_1_002_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,CA,...,0,0,0,0,3,4,3.970703,0.137085,0.107117,0
3,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,CA_1,1914,0,HOBBIES_1_002_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,CA,...,0,0,0,0,3,4,3.970703,0.137085,0.107117,0
4,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,CA_1,1914,0,HOBBIES_1_003_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,CA,...,0,0,0,0,3,4,2.970703,0.379150,0.285645,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189035,FOODS_3_825_WI_3_evaluation,FOODS_3_825,WI_3,1941,0,FOODS_3_825_WI_3_evaluation,d_1941,FOODS_3,FOODS,WI,...,0,0,0,0,2,5,3.980469,1.377930,0.714355,2
189036,FOODS_3_826_WI_3_validation,FOODS_3_826,WI_3,1941,0,FOODS_3_826_WI_3_evaluation,d_1941,FOODS_3,FOODS,WI,...,0,0,0,0,2,5,1.280273,1.388672,1.952148,0
189037,FOODS_3_826_WI_3_evaluation,FOODS_3_826,WI_3,1941,0,FOODS_3_826_WI_3_evaluation,d_1941,FOODS_3,FOODS,WI,...,0,0,0,0,2,5,1.280273,1.388672,1.952148,0
189038,FOODS_3_827_WI_3_validation,FOODS_3_827,WI_3,1941,0,FOODS_3_827_WI_3_evaluation,d_1941,FOODS_3,FOODS,WI,...,0,0,0,0,2,5,1.000000,2.166016,1.809570,1


In [3]:
#Export the processed submission file
validation_df.to_csv(export_path + validation_file)

In [4]:
#Clear up variables and free up memory 
del(sales_train_validation_processing, submission_file, chunk, validation_df)

# BUILD MODELS and PREDICT

### Prepare the training and validation dataframes

In [5]:
#We will Copy and paste the contents of this cell before each model
#since it is unnecessary to run the whole notebook eachtime we restart the kernel.
#This cell has the information needed to get things started and consistent.

###################################################################################################
import pickle
import pandas as pd

#import sample_submission.csv from
sample_submission_path = 'data/sample_submission.csv'

#import sell_price.csv from
sell_prices_path = 'data/sell_prices.csv'

#import sales_train_validation.csv from
#sales_train_validation_path = 'data/sales_train_validation.csv'

#import sales_train_evaluation.csv from
sales_train_validation_path = 'data/sales_train_evaluation.csv'

#import calendar.csv from
calendar_path = 'data/calendar.csv'

#############################################
#Export files to folder
export_path = 'data/outputs/'

#name of the final preprocessed file to be exported
final_preprocessed_file = 'sales_train_evaluation_preprocessed_final.csv'  #input('Enter the name of the final preprocessed file: ')

#name of the preprocessed validation file to be exported
validation_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_prediction.csv'

#name of the preprocessed evaluation_data to be exported
evaluation_file = 'submission_kaggle_preprocessed_to_be_used_4_private_LB_prediction.csv'

#name of the file to be exported and used for submission
submission_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_submission.csv'

###################################################

#Store the datatypes prior to export to be able to use later when importing
schema={'id': 'O', 'item_id': 'O', 'dept_id': 'O', 'cat_id': 'O', 'store_id': 'O',
        'state_id': 'O', 'FOODS_2': 'int8', 'FOODS_3': 'int8', 'HOBBIES_1': 'int8',
        'HOBBIES_2': 'int8', 'HOUSEHOLD_1': 'int8', 'HOUSEHOLD_2': 'int8',
        'HOBBIES': 'int8', 'HOUSEHOLD': 'int8', 'TX': 'int8', 'WI': 'int8',
        'CA_2': 'int8', 'CA_3': 'int8', 'CA_4': 'int8', 'TX_1': 'int8', 'TX_2': 'int8',
        'TX_3': 'int8', 'WI_1': 'int8', 'WI_2': 'int8', 'WI_3': 'int8', 'd': 'O',
        'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int8', 'month': 'int8',
        'year': 'int16', 'snap_CA': 'int8', 'snap_TX': 'int8', 'snap_WI': 'int8',
        'day': 'int16', 'EidAlAdha': 'int8', 'OrthodoxEaster': 'int8', 'LaborDay': 'int8',
        'Chanukah End': 'int8', 'LentStart': 'int8', 'Halloween': 'int8', 'NBAFinalsEnd': 'int8',
        'Cinco De Mayo': 'int8', 'LentWeek2': 'int8', 'ColumbusDay': 'int8', 'Pesach End': 'int8',
        'OrthodoxChristmas': 'int8', 'Eid al-Fitr': 'int8', 'Easter': 'int8', "Father's day": 'int8',
        'NBAFinalsStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
        'SuperBowl': 'int8', 'Ramadan starts': 'int8', 'MemorialDay': 'int8', 'PresidentsDay': 'int8',
        'NewYear': 'int8', 'Thanksgiving': 'int8', 'Christmas': 'int8', 'Purim End': 'int8',
        "Mother's day": 'int8', 'StPatricksDay': 'int8', 'IndependenceDay': 'int8', 'VeteransDay': 'int8',
        'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'Sporting': 'int8',
        'sell_price': 'float16', 'weekly_avg_4weeks_ago': 'float16', 'last_month_avg': 'float16'}

#identify the columns necessary when importing preprocessed file
training_cols = organized_cols = [
         'id', 'day', 'item_id', 'store_id','wm_yr_wk', 'year',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago', 

         'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]


#save the submission file column names to a variable
submission_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']
##############################################################################################

In [6]:
import pandas as pd

#read in sales_train_validation_process 
training_df_chunks = pd.read_csv(export_path+final_preprocessed_file, usecols=training_cols, dtype=schema, chunksize=100000)

training_df = pd.DataFrame()
for chunk in training_df_chunks:
    #drop missing values
    chunk.dropna(subset=['sell_price', 'last_month_avg', 'weekly_avg_4weeks_ago'], inplace=True)
    
    #filter chunk for days between d_1 and d_1913
    chunk = chunk .loc[chunk.day<1914]
    
    #add chunk to training data
    training_df = pd.concat([training_df, chunk])
   

In [7]:
#and read the validation_df data
validation_df = pd.read_csv(export_path+validation_file, dtype=schema)

In [8]:
#Check for missing values
training_df.isna().any()

id                       False
day                      False
item_id                  False
store_id                 False
wm_yr_wk                 False
                         ...  
month                    False
sell_price               False
last_month_avg           False
weekly_avg_4weeks_ago    False
quantity_sold            False
Length: 68, dtype: bool

In [9]:
#Check for missing values
validation_df.isna().any()

Unnamed: 0               False
id                       False
item_id                  False
store_id                 False
day                      False
                         ...  
month                    False
sell_price               False
last_month_avg           False
weekly_avg_4weeks_ago    False
quantity_sold_y          False
Length: 75, dtype: bool

## MODEL 1: SGD ML Multiple Linear Regression item by item training

#### This is the baseline Gradient Descent Machine learning model for multiple linear regression
#### This is an example for overfitted model with item by item training. The RMSE is likely to be much higher with the test data reagardless how well the algorithm is working. Partial fitment is a better a method for large dataset.

#### Hyperparameters are chosen to reduce the time it takes to complete the experiment. 

#### For better results, adjust the learning rate with a scheduled LR model, adjust the alpha value, early stopping threshold, until algorithm starts learning.

In [10]:
import time
x=time.time()  #record the start time

weight_dict = {}  #this will store the best weights for each item

#make a list of the unique item_ids
items = validation_df.item_id.unique()

#Set parameters
epochs = 50
early_stop_threshold = 3
learning_rate = 0.01
alpha = 0.01
verbose = True

#iterate over items 
for item in items:  #calculate the weights for each item by iterating over each item
    
    
    temp_df = training_df[training_df['item_id'] == item]

    X = temp_df[input_cols]   #filter the dataset on the item_id

    target = np.array(temp_df[target_col]).reshape(-1,1)   #labels



    #initialize coefficients and the bias
    np.random.seed(10)
    W = np.random.rand(62, 1)/60
    B = np.random.random(size=(1))/100
    N = np.random.rand(X.shape[0],1)/30

    #count the number of items
    num_observations = X.shape[0]

    #initialize the loss and and a loss list to help with early stopping
    losses = [0]
    loss_score = 0


    for epoch in range(epochs):

        Y =  X.dot(W) +  B + N #calculate the targets

        deltas = target - Y  #Find the error

        loss = deltas.pow(2).mean().pow(0.5)   #Loss function
        
        #set up an early stopping mechanism
        #add the loss to the losses list
        
        if losses[-1] < loss[0]:

            loss_score += 1   #keep track of the change in loss function
            
            losses.append(loss[0])

        if verbose==True:
            print(f'Epoch: {epoch}  Loss: {loss[0]:.3f}')


        if loss_score >= early_stop_threshold or epoch == (epochs-1):   #If the loss is increasing  or ran out of epochs stop learning

            weight_dict[item] = (W,B)
            
                      

            break

        #scale the deltas
        deltas_scaled = deltas / num_observations

        #Optimize
        #update the weights biases
        w_momentum = alpha * learning_rate * np.dot(X.T, deltas_scaled)
        W = W - learning_rate * np.dot(X.T, deltas_scaled) - w_momentum
        
        b_momentum = alpha * learning_rate * np.sum(deltas_scaled)
        B = B - learning_rate * np.sum(deltas_scaled) - b_momentum
print('Completed...')

Epoch: 0  Loss: 25.732
Epoch: 1  Loss: 1054249.733
Epoch: 2  Loss: 43210877524.188
Epoch: 0  Loss: 25.824
Epoch: 1  Loss: 1056964.577
Epoch: 2  Loss: 43272830582.743
Epoch: 0  Loss: 25.918
Epoch: 1  Loss: 1062441.533
Epoch: 2  Loss: 43560395070.051
Epoch: 0  Loss: 24.219
Epoch: 1  Loss: 985440.630
Epoch: 2  Loss: 40342934314.279
Epoch: 0  Loss: 25.320
Epoch: 1  Loss: 1035437.102
Epoch: 2  Loss: 42393637545.495
Epoch: 0  Loss: 25.276
Epoch: 1  Loss: 1033121.184
Epoch: 2  Loss: 42316731912.524
Epoch: 0  Loss: 25.896
Epoch: 1  Loss: 1060734.293
Epoch: 2  Loss: 43456320840.342
Epoch: 0  Loss: 22.666
Epoch: 1  Loss: 880849.764
Epoch: 2  Loss: 36062547666.027
Epoch: 0  Loss: 25.313
Epoch: 1  Loss: 1033890.639
Epoch: 2  Loss: 42329150918.334
Epoch: 0  Loss: 25.497
Epoch: 1  Loss: 1043224.979
Epoch: 2  Loss: 42708044661.457
Epoch: 0  Loss: 25.991
Epoch: 1  Loss: 1064546.334
Epoch: 2  Loss: 43606208465.819
Epoch: 0  Loss: 25.736
Epoch: 1  Loss: 1053212.317
Epoch: 2  Loss: 43117258413.547
Epoch:

Epoch: 0  Loss: 25.851
Epoch: 1  Loss: 1059763.143
Epoch: 2  Loss: 43457883665.603
Epoch: 0  Loss: 25.478
Epoch: 1  Loss: 1042291.002
Epoch: 2  Loss: 42670186121.538
Epoch: 0  Loss: 25.881
Epoch: 1  Loss: 1059286.792
Epoch: 2  Loss: 43365555242.324
Epoch: 0  Loss: 25.874
Epoch: 1  Loss: 1059681.992
Epoch: 2  Loss: 43409396182.520
Epoch: 0  Loss: 25.182
Epoch: 1  Loss: 1029110.600
Epoch: 2  Loss: 42132543600.983
Epoch: 0  Loss: 25.616
Epoch: 1  Loss: 1048191.237
Epoch: 2  Loss: 42916634994.121
Epoch: 0  Loss: 25.936
Epoch: 1  Loss: 1061640.519
Epoch: 2  Loss: 43462027183.995
Epoch: 0  Loss: 25.714
Epoch: 1  Loss: 1051955.721
Epoch: 2  Loss: 43065935801.714
Epoch: 0  Loss: 25.994
Epoch: 1  Loss: 1064063.291
Epoch: 2  Loss: 43561108662.652
Epoch: 0  Loss: 25.888
Epoch: 1  Loss: 1060777.539
Epoch: 2  Loss: 43475056872.645
Epoch: 0  Loss: 25.899
Epoch: 1  Loss: 1060635.146
Epoch: 2  Loss: 43444019609.486
Epoch: 0  Loss: 24.751
Epoch: 1  Loss: 1005851.883
Epoch: 2  Loss: 41214437966.823
Epoc

Epoch: 0  Loss: 25.975
Epoch: 1  Loss: 1063879.482
Epoch: 2  Loss: 43579247978.454
Epoch: 0  Loss: 25.388
Epoch: 1  Loss: 1040246.904
Epoch: 2  Loss: 42661034866.333
Epoch: 0  Loss: 25.793
Epoch: 1  Loss: 1055581.016
Epoch: 2  Loss: 43213860157.443
Epoch: 0  Loss: 25.639
Epoch: 1  Loss: 1049873.712
Epoch: 2  Loss: 43022980421.689
Epoch: 0  Loss: 23.983
Epoch: 1  Loss: 964217.547
Epoch: 2  Loss: 39473644314.093
Epoch: 0  Loss: 25.545
Epoch: 1  Loss: 1045343.920
Epoch: 2  Loss: 42815527876.243
Epoch: 0  Loss: 25.006
Epoch: 1  Loss: 1022543.475
Epoch: 2  Loss: 41925962494.546
Epoch: 0  Loss: 26.046
Epoch: 1  Loss: 1066612.099
Epoch: 2  Loss: 43681163481.754
Epoch: 0  Loss: 25.632
Epoch: 1  Loss: 1048777.197
Epoch: 2  Loss: 42935988092.360
Epoch: 0  Loss: 25.996
Epoch: 1  Loss: 1064532.876
Epoch: 2  Loss: 43597615141.867
Epoch: 0  Loss: 25.684
Epoch: 1  Loss: 1050911.635
Epoch: 2  Loss: 43023852064.974
Epoch: 0  Loss: 25.373
Epoch: 1  Loss: 1036835.094
Epoch: 2  Loss: 42449721104.210
Epoch

Epoch: 0  Loss: 25.834
Epoch: 1  Loss: 1058197.942
Epoch: 2  Loss: 43359658283.182
Epoch: 0  Loss: 25.946
Epoch: 1  Loss: 1062683.369
Epoch: 2  Loss: 43529859186.554
Epoch: 0  Loss: 25.682
Epoch: 1  Loss: 1051626.168
Epoch: 2  Loss: 43083496706.403
Epoch: 0  Loss: 25.729
Epoch: 1  Loss: 1052805.031
Epoch: 2  Loss: 43100668257.122
Epoch: 0  Loss: 25.618
Epoch: 1  Loss: 1048142.759
Epoch: 2  Loss: 42909575491.739
Epoch: 0  Loss: 26.019
Epoch: 1  Loss: 1065160.018
Epoch: 2  Loss: 43611534854.306
Epoch: 0  Loss: 23.759
Epoch: 1  Loss: 947139.806
Epoch: 2  Loss: 38774551168.639
Epoch: 0  Loss: 25.520
Epoch: 1  Loss: 1044061.395
Epoch: 2  Loss: 42743501159.255
Epoch: 0  Loss: 25.976
Epoch: 1  Loss: 1063296.383
Epoch: 2  Loss: 43529840321.318
Epoch: 0  Loss: 25.925
Epoch: 1  Loss: 1061189.935
Epoch: 2  Loss: 43444220108.921
Epoch: 0  Loss: 25.243
Epoch: 1  Loss: 1030518.415
Epoch: 2  Loss: 42216478952.033
Epoch: 0  Loss: 25.599
Epoch: 1  Loss: 1047468.124
Epoch: 2  Loss: 42882278619.656
Epoch

Epoch: 0  Loss: 25.927
Epoch: 1  Loss: 1061304.413
Epoch: 2  Loss: 43452383571.619
Epoch: 0  Loss: 25.869
Epoch: 1  Loss: 1058765.842
Epoch: 2  Loss: 43344323659.932
Epoch: 0  Loss: 25.306
Epoch: 1  Loss: 1034839.461
Epoch: 2  Loss: 42364746618.742
Epoch: 0  Loss: 24.844
Epoch: 1  Loss: 1005806.521
Epoch: 2  Loss: 41176146131.530
Epoch: 0  Loss: 25.536
Epoch: 1  Loss: 1044773.640
Epoch: 2  Loss: 42771690143.683
Epoch: 0  Loss: 25.943
Epoch: 1  Loss: 1063318.575
Epoch: 2  Loss: 43591809481.128
Epoch: 0  Loss: 25.616
Epoch: 1  Loss: 1048167.391
Epoch: 2  Loss: 42910766978.057
Epoch: 0  Loss: 25.534
Epoch: 1  Loss: 1045244.617
Epoch: 2  Loss: 42823809593.929
Epoch: 0  Loss: 25.416
Epoch: 1  Loss: 1039276.563
Epoch: 2  Loss: 42546660224.296
Epoch: 0  Loss: 25.693
Epoch: 1  Loss: 1051172.382
Epoch: 2  Loss: 43033917345.756
Epoch: 0  Loss: 25.411
Epoch: 1  Loss: 1039411.839
Epoch: 2  Loss: 42552318936.809
Epoch: 0  Loss: 25.095
Epoch: 1  Loss: 1026318.124
Epoch: 2  Loss: 42022192782.576
Epoc

...

Epoch: 0  Loss: 25.331
Epoch: 1  Loss: 1037076.672
Epoch: 2  Loss: 42507305532.062
Epoch: 0  Loss: 23.954
Epoch: 1  Loss: 969175.058
Epoch: 2  Loss: 39676581358.909
Epoch: 0  Loss: 24.950
Epoch: 1  Loss: 1019669.886
Epoch: 2  Loss: 41744094421.555
Epoch: 0  Loss: 25.412
Epoch: 1  Loss: 1039226.287
Epoch: 2  Loss: 42544454658.568
Epoch: 0  Loss: 25.676
Epoch: 1  Loss: 1051703.910
Epoch: 2  Loss: 43101136354.025
Epoch: 0  Loss: 25.800
Epoch: 1  Loss: 1057308.731
Epoch: 2  Loss: 43346409919.907
Epoch: 0  Loss: 25.342
Epoch: 1  Loss: 1034265.500
Epoch: 2  Loss: 42341530238.905
Epoch: 0  Loss: 24.989
Epoch: 1  Loss: 1022546.186
Epoch: 2  Loss: 41905198546.334
Epoch: 0  Loss: 19.113
Epoch: 1  Loss: 594765.177
Epoch: 2  Loss: 24372171594.656
Epoch: 0  Loss: 22.444
Epoch: 1  Loss: 895579.287
Epoch: 2  Loss: 36666538484.575
Epoch: 0  Loss: 25.731
Epoch: 1  Loss: 1054321.511
Epoch: 2  Loss: 43226548603.332
Epoch: 0  Loss: 25.620
Epoch: 1  Loss: 1048549.005
Epoch: 2  Loss: 42938891317.839
Epoch: 

Epoch: 0  Loss: 23.081
Epoch: 1  Loss: 804630.996
Epoch: 2  Loss: 32956727736.927
Epoch: 0  Loss: 20.643
Epoch: 1  Loss: 717755.805
Epoch: 2  Loss: 29398817878.579
Epoch: 0  Loss: 25.563
Epoch: 1  Loss: 1046616.751
Epoch: 2  Loss: 42874842662.675
Epoch: 0  Loss: 25.703
Epoch: 1  Loss: 1052118.701
Epoch: 2  Loss: 43094163486.251
Epoch: 0  Loss: 18.493
Epoch: 1  Loss: 720042.157
Epoch: 2  Loss: 29478758641.605
Epoch: 0  Loss: 23.001
Epoch: 1  Loss: 927289.914
Epoch: 2  Loss: 37986814028.365
Epoch: 0  Loss: 23.796
Epoch: 1  Loss: 967138.345
Epoch: 2  Loss: 39593230604.540
Epoch: 0  Loss: 24.445
Epoch: 1  Loss: 994879.678
Epoch: 2  Loss: 40728839751.991
Epoch: 0  Loss: 19.627
Epoch: 1  Loss: 663401.975
Epoch: 2  Loss: 27159965146.239
Epoch: 0  Loss: 24.703
Epoch: 1  Loss: 1006292.446
Epoch: 2  Loss: 41222668284.625
Epoch: 0  Loss: 25.286
Epoch: 1  Loss: 1033647.159
Epoch: 2  Loss: 42315973932.047
Epoch: 0  Loss: 25.547
Epoch: 1  Loss: 1044199.183
Epoch: 2  Loss: 42747857611.469
Epoch: 0  L

Epoch: 0  Loss: 25.754
Epoch: 1  Loss: 1053893.352
Epoch: 2  Loss: 43145628145.232
Epoch: 0  Loss: 18.473
Epoch: 1  Loss: 42098.684
Epoch: 2  Loss: 1724013319.706
Epoch: 0  Loss: 25.769
Epoch: 1  Loss: 1054536.140
Epoch: 2  Loss: 43175271288.004
Epoch: 0  Loss: 24.702
Epoch: 1  Loss: 1007082.768
Epoch: 2  Loss: 41286911493.708
Epoch: 0  Loss: 25.594
Epoch: 1  Loss: 1047196.463
Epoch: 2  Loss: 42873424958.609
Epoch: 0  Loss: 25.007
Epoch: 1  Loss: 1022402.735
Epoch: 2  Loss: 41855616179.845
Epoch: 0  Loss: 25.293
Epoch: 1  Loss: 1034530.043
Epoch: 2  Loss: 42379198725.807
Epoch: 0  Loss: 23.342
Epoch: 1  Loss: 951244.429
Epoch: 2  Loss: 38988954845.511
Epoch: 0  Loss: 23.816
Epoch: 1  Loss: 971108.691
Epoch: 2  Loss: 39765689451.594
Epoch: 0  Loss: 25.665
Epoch: 1  Loss: 1052109.683
Epoch: 2  Loss: 43158397559.039
Epoch: 0  Loss: 25.601
Epoch: 1  Loss: 1046819.209
Epoch: 2  Loss: 42855151292.622
Epoch: 0  Loss: 24.823
Epoch: 1  Loss: 1014206.122
Epoch: 2  Loss: 41520468815.441
Epoch: 0 

Epoch: 0  Loss: 23.400
Epoch: 1  Loss: 945691.198
Epoch: 2  Loss: 38718664477.005
Epoch: 0  Loss: 24.091
Epoch: 1  Loss: 978697.435
Epoch: 2  Loss: 40066518592.564
Epoch: 0  Loss: 25.053
Epoch: 1  Loss: 1023727.520
Epoch: 2  Loss: 41909888600.164
Epoch: 0  Loss: 24.579
Epoch: 1  Loss: 1001522.570
Epoch: 2  Loss: 41000861361.769
Epoch: 0  Loss: 25.400
Epoch: 1  Loss: 1040006.053
Epoch: 2  Loss: 42621951020.514
Epoch: 0  Loss: 25.064
Epoch: 1  Loss: 1024416.427
Epoch: 2  Loss: 41953022954.704
Epoch: 0  Loss: 24.975
Epoch: 1  Loss: 1018927.504
Epoch: 2  Loss: 41713527809.631
Epoch: 0  Loss: 22.616
Epoch: 1  Loss: 919262.008
Epoch: 2  Loss: 37692227508.115
Epoch: 0  Loss: 25.595
Epoch: 1  Loss: 1048358.470
Epoch: 2  Loss: 42979650999.205
Epoch: 0  Loss: 25.364
Epoch: 1  Loss: 1036283.884
Epoch: 2  Loss: 42423823316.966
Epoch: 0  Loss: 23.322
Epoch: 1  Loss: 927596.504
Epoch: 2  Loss: 37974822851.220
Epoch: 0  Loss: 24.282
Epoch: 1  Loss: 983872.780
Epoch: 2  Loss: 40317445910.432
Epoch: 0 

Epoch: 0  Loss: 23.638
Epoch: 1  Loss: 943847.488
Epoch: 2  Loss: 38639827550.693
Epoch: 0  Loss: 25.798
Epoch: 1  Loss: 1056252.300
Epoch: 2  Loss: 43263237274.321
Epoch: 0  Loss: 24.870
Epoch: 1  Loss: 1015491.367
Epoch: 2  Loss: 41626213033.363
Epoch: 0  Loss: 20.932
Epoch: 1  Loss: 824067.379
Epoch: 2  Loss: 33802875704.410
Epoch: 0  Loss: 24.752
Epoch: 1  Loss: 1011433.171
Epoch: 2  Loss: 41449912675.690
Epoch: 0  Loss: 25.766
Epoch: 1  Loss: 1054477.033
Epoch: 2  Loss: 43171219895.291
Epoch: 0  Loss: 25.726
Epoch: 1  Loss: 1052551.001
Epoch: 2  Loss: 43090188191.556
Epoch: 0  Loss: 19.332
Epoch: 1  Loss: 703268.033
Epoch: 2  Loss: 28799483302.491
Epoch: 0  Loss: 25.008
Epoch: 1  Loss: 1022478.999
Epoch: 2  Loss: 41858842412.346
Epoch: 0  Loss: 25.365
Epoch: 1  Loss: 1037278.607
Epoch: 2  Loss: 42464592538.973
Epoch: 0  Loss: 22.334
Epoch: 1  Loss: 859351.043
Epoch: 2  Loss: 35182439292.024
Epoch: 0  Loss: 24.059
Epoch: 1  Loss: 981642.797
Epoch: 2  Loss: 40249212141.187
Epoch: 0 

In [11]:
#pickle the weights
import pickle
with open(export_path+'M5_Multiple_Linear_Regression_Baseline_Weights_Dictionary.pickle', 'wb') as f:
    pickle.dump(weight_dict, f)

#### Make the predictions

In [12]:
###################################################################################################
import pickle
import pandas as pd

#import sample_submission.csv from
sample_submission_path = 'data/sample_submission.csv'

#import sell_price.csv from
sell_prices_path = 'data/sell_prices.csv'

#import sales_train_validation.csv from
#sales_train_validation_path = 'data/sales_train_validation.csv'

#import sales_train_evaluation.csv from
sales_train_validation_path = 'data/sales_train_evaluation.csv'

#import calendar.csv from
calendar_path = 'data/calendar.csv'

#############################################
#Export files to folder
export_path = 'data/outputs/'

#name of the final preprocessed file to be exported
final_preprocessed_file = 'sales_train_evaluation_preprocessed_final.csv'  #input('Enter the name of the final preprocessed file: ')

#name of the preprocessed validation file to be exported
validation_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_prediction.csv'

#name of the preprocessed evaluation_data to be exported
evaluation_file = 'submission_kaggle_preprocessed_to_be_used_4_private_LB_prediction.csv'

#name of the file to be exported and used for submission
submission_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_submission.csv'

###################################################

#Store the datatypes prior to export to be able to use later when importing
schema={'id': 'O', 'item_id': 'O', 'dept_id': 'O', 'cat_id': 'O', 'store_id': 'O',
        'state_id': 'O', 'FOODS_2': 'int8', 'FOODS_3': 'int8', 'HOBBIES_1': 'int8',
        'HOBBIES_2': 'int8', 'HOUSEHOLD_1': 'int8', 'HOUSEHOLD_2': 'int8',
        'HOBBIES': 'int8', 'HOUSEHOLD': 'int8', 'TX': 'int8', 'WI': 'int8',
        'CA_2': 'int8', 'CA_3': 'int8', 'CA_4': 'int8', 'TX_1': 'int8', 'TX_2': 'int8',
        'TX_3': 'int8', 'WI_1': 'int8', 'WI_2': 'int8', 'WI_3': 'int8', 'd': 'O',
        'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int8', 'month': 'int8',
        'year': 'int16', 'snap_CA': 'int8', 'snap_TX': 'int8', 'snap_WI': 'int8',
        'day': 'int16', 'EidAlAdha': 'int8', 'OrthodoxEaster': 'int8', 'LaborDay': 'int8',
        'Chanukah End': 'int8', 'LentStart': 'int8', 'Halloween': 'int8', 'NBAFinalsEnd': 'int8',
        'Cinco De Mayo': 'int8', 'LentWeek2': 'int8', 'ColumbusDay': 'int8', 'Pesach End': 'int8',
        'OrthodoxChristmas': 'int8', 'Eid al-Fitr': 'int8', 'Easter': 'int8', "Father's day": 'int8',
        'NBAFinalsStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
        'SuperBowl': 'int8', 'Ramadan starts': 'int8', 'MemorialDay': 'int8', 'PresidentsDay': 'int8',
        'NewYear': 'int8', 'Thanksgiving': 'int8', 'Christmas': 'int8', 'Purim End': 'int8',
        "Mother's day": 'int8', 'StPatricksDay': 'int8', 'IndependenceDay': 'int8', 'VeteransDay': 'int8',
        'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'Sporting': 'int8',
        'sell_price': 'float16', 'weekly_avg_4weeks_ago': 'float16', 'last_month_avg': 'float16'}

#identify the columns necessary when importing preprocessed file
training_cols = organized_cols = [
         'id', 'day', 'item_id', 'store_id','wm_yr_wk', 'year',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago', 

         'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]


#save the submission file column names to a variable
submission_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']
##############################################################################################



with open('data/outputs/Model_1_Multiple_Linear_Regression_Baseline_Weights_Dictionary.pickle', 'rb') as f:
    weight_dict = pickle.load(f)
    
validation_df = pd.read_csv(export_path+validation_file, dtype=schema)

    
#Prepare the submission file
validation_df['predictions'] = 0

for i in range(len(validation_df)):
    
    x = validation_df.iloc[i][input_cols]
    
    item = validation_df.loc[i, 'item_id']
    
    #dot product of the inputs with weights and additon of bias gives the prediction for each item
    validation_df.loc[i, 'predictions'] = x.dot(weight_dict[item][0]) + weight_dict[item][1][0]
    
kaggle_output_piv = validation_df.pivot_table(index='id', values='predictions', columns='day')
kaggle_output_piv.columns = submission_columns

#to export in submission format uncomment below
kaggle_output_piv.to_csv(export_path+'model_1_submission_ML_MR_result_to_be_used_4submission.csv')
kaggle_output_piv

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_evaluation,4.267933e+10,4.267937e+10,4.267941e+10,4.267946e+10,4.267950e+10,4.267924e+10,4.267936e+10,4.267940e+10,4.267945e+10,4.267948e+10,...,4.267957e+10,4.267932e+10,4.267937e+10,4.267940e+10,4.267944e+10,4.267949e+10,4.267953e+10,4.267957e+10,4.267931e+10,4.267936e+10
FOODS_1_001_CA_1_validation,4.267933e+10,4.267937e+10,4.267941e+10,4.267946e+10,4.267950e+10,4.267924e+10,4.267936e+10,4.267940e+10,4.267945e+10,4.267948e+10,...,4.267957e+10,4.267932e+10,4.267937e+10,4.267940e+10,4.267944e+10,4.267949e+10,4.267953e+10,4.267957e+10,4.267931e+10,4.267936e+10
FOODS_1_001_CA_2_evaluation,4.267933e+10,4.267937e+10,4.267942e+10,4.267946e+10,4.267950e+10,4.267924e+10,4.267936e+10,4.267940e+10,4.267945e+10,4.267948e+10,...,4.267957e+10,4.267932e+10,4.267937e+10,4.267940e+10,4.267945e+10,4.267949e+10,4.267953e+10,4.267957e+10,4.267931e+10,4.267936e+10
FOODS_1_001_CA_2_validation,4.267933e+10,4.267937e+10,4.267942e+10,4.267946e+10,4.267950e+10,4.267924e+10,4.267936e+10,4.267940e+10,4.267945e+10,4.267948e+10,...,4.267957e+10,4.267932e+10,4.267937e+10,4.267940e+10,4.267945e+10,4.267949e+10,4.267953e+10,4.267957e+10,4.267931e+10,4.267936e+10
FOODS_1_001_CA_3_evaluation,4.267933e+10,4.267937e+10,4.267942e+10,4.267946e+10,4.267950e+10,4.267924e+10,4.267936e+10,4.267940e+10,4.267945e+10,4.267948e+10,...,4.267957e+10,4.267932e+10,4.267937e+10,4.267940e+10,4.267945e+10,4.267949e+10,4.267953e+10,4.267957e+10,4.267931e+10,4.267936e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_WI_1_validation,4.354197e+10,4.354201e+10,4.354206e+10,4.354210e+10,4.354214e+10,4.354188e+10,4.354200e+10,4.354205e+10,4.354209e+10,4.354213e+10,...,4.354221e+10,4.354196e+10,4.354200e+10,4.354204e+10,4.354208e+10,4.354213e+10,4.354217e+10,4.354221e+10,4.354195e+10,4.354200e+10
HOUSEHOLD_2_516_WI_2_evaluation,4.354197e+10,4.354201e+10,4.354206e+10,4.354210e+10,4.354214e+10,4.354188e+10,4.354200e+10,4.354205e+10,4.354209e+10,4.354213e+10,...,4.354221e+10,4.354196e+10,4.354200e+10,4.354204e+10,4.354208e+10,4.354213e+10,4.354217e+10,4.354221e+10,4.354195e+10,4.354200e+10
HOUSEHOLD_2_516_WI_2_validation,4.354197e+10,4.354201e+10,4.354206e+10,4.354210e+10,4.354214e+10,4.354188e+10,4.354200e+10,4.354205e+10,4.354209e+10,4.354213e+10,...,4.354221e+10,4.354196e+10,4.354200e+10,4.354204e+10,4.354208e+10,4.354213e+10,4.354217e+10,4.354221e+10,4.354195e+10,4.354200e+10


Rank: 4182
DataWise
Caleb Bak contributor tier
Best Score: 0.90096

Your submission scored 6.35484, which is not an improvement of your best score. Keep trying!

#### One of the reasons for a poor RMSE score is training the model item by item. When the chunks were random, rather than filtered by item_id, RMSE score was 2.5.

## MODEL 2: sklearn.SGDRegressor Model

#### This is a linear multiple regression model using the sklearn library. It is much more straighforward than hardcoding the ML algorithm. Sklearn's SGDRegressor allows partial fitment. Therefore, we can read the large datasets in chunks and keep using each chunk to train the model until all data is read.

In [13]:
from sklearn.linear_model import SGDRegressor
import pickle
import pandas as pd
import numpy as np
#################################################################################
#############################################
#Export files to folder
export_path = 'data/outputs/'

#name of the final preprocessed file to be exported
final_preprocessed_file = 'sales_train_evaluation_preprocessed_final.csv'  #input('Enter the name of the final preprocessed file: ')

#name of the preprocessed validation file to be exported
validation_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_prediction.csv'

#name of the preprocessed evaluation_data to be exported
evaluation_file = 'submission_kaggle_preprocessed_to_be_used_4_private_LB_prediction.csv'

#name of the file to be exported and used for submission
submission_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_submission.csv'

###################################################

#Store the datatypes prior to export to be able to use later when importing
schema={'id': 'O', 'item_id': 'O', 'dept_id': 'O', 'cat_id': 'O', 'store_id': 'O',
        'state_id': 'O', 'FOODS_2': 'int8', 'FOODS_3': 'int8', 'HOBBIES_1': 'int8',
        'HOBBIES_2': 'int8', 'HOUSEHOLD_1': 'int8', 'HOUSEHOLD_2': 'int8',
        'HOBBIES': 'int8', 'HOUSEHOLD': 'int8', 'TX': 'int8', 'WI': 'int8',
        'CA_2': 'int8', 'CA_3': 'int8', 'CA_4': 'int8', 'TX_1': 'int8', 'TX_2': 'int8',
        'TX_3': 'int8', 'WI_1': 'int8', 'WI_2': 'int8', 'WI_3': 'int8', 'd': 'O',
        'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int8', 'month': 'int8',
        'year': 'int16', 'snap_CA': 'int8', 'snap_TX': 'int8', 'snap_WI': 'int8',
        'day': 'int16', 'EidAlAdha': 'int8', 'OrthodoxEaster': 'int8', 'LaborDay': 'int8',
        'Chanukah End': 'int8', 'LentStart': 'int8', 'Halloween': 'int8', 'NBAFinalsEnd': 'int8',
        'Cinco De Mayo': 'int8', 'LentWeek2': 'int8', 'ColumbusDay': 'int8', 'Pesach End': 'int8',
        'OrthodoxChristmas': 'int8', 'Eid al-Fitr': 'int8', 'Easter': 'int8', "Father's day": 'int8',
        'NBAFinalsStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
        'SuperBowl': 'int8', 'Ramadan starts': 'int8', 'MemorialDay': 'int8', 'PresidentsDay': 'int8',
        'NewYear': 'int8', 'Thanksgiving': 'int8', 'Christmas': 'int8', 'Purim End': 'int8',
        "Mother's day": 'int8', 'StPatricksDay': 'int8', 'IndependenceDay': 'int8', 'VeteransDay': 'int8',
        'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'Sporting': 'int8',
        'sell_price': 'float16', 'weekly_avg_4weeks_ago': 'float16', 'last_month_avg': 'float16'}

#identify the columns necessary when importing preprocessed file
training_cols = organized_cols = [
         'id', 'day', 'item_id', 'store_id','wm_yr_wk', 'year',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago', 

         'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]


#save the submission file column names to a variable
submission_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']


#################################################################################
 
training_df = pd.read_csv(export_path + final_preprocessed_file, usecols=training_cols, dtype=schema,chunksize=100000)
validation_df = pd.read_csv(export_path+validation_file, dtype=schema)
    
#Prepare the submission file
validation_df['predictions'] = 0

#initiate the model
model = SGDRegressor()

#train the model
for chunk in training_df:
    try:
        chunk.dropna(inplace=True)
        model.partial_fit(chunk[input_cols], chunk[target_col])
    except:
        continue
    
#Make predictions
validation_df['predictions'] = model.predict(validation_df[input_cols])

In [14]:
#Format the results for submission
kaggle_output_piv = validation_df.pivot_table(index='id', values='predictions', columns='day')
kaggle_output_piv.columns = submission_columns

#negative valuess should be zero and round  predictions
for col in kaggle_output_piv:
    kaggle_output_piv[kaggle_output_piv[col]<0]=0
kaggle_output_piv

#to export in submission format uncomment below
kaggle_output_piv.to_csv(export_path+'Model_2_submission_sk_SGDRegressor_result_to_be_used_4submission.csv')
kaggle_output_piv


,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FOODS_1_001_CA_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FOODS_1_001_CA_2_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FOODS_1_001_CA_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FOODS_1_001_CA_3_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_WI_1_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOUSEHOLD_2_516_WI_2_evaluation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HOUSEHOLD_2_516_WI_2_validation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Kaggle score
DataWise
Caleb Bak contributor tier
best score: 0.90096
Your submission scored 5.44561, which is not an improvement of your best score. Keep trying!

In [15]:
validation_df

,Unnamed: 0,id,item_id,store_id,day,quantity_sold,id_y,d,dept_id,cat_id,...,Cultural,National,Religious,wday,month,sell_price,last_month_avg,weekly_avg_4weeks_ago,quantity_sold_y,predictions
0,0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,CA_1,1914,0,HOBBIES_1_001_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,...,0,0,0,3,4,8.382812,0.991699,0.928711,0,-4.436493e+14
1,1,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,CA_1,1914,0,HOBBIES_1_001_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,...,0,0,0,3,4,8.382812,0.991699,0.928711,0,-4.436493e+14
2,2,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,CA_1,1914,0,HOBBIES_1_002_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,...,0,0,0,3,4,3.970703,0.137085,0.107117,0,-4.608542e+14
3,3,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,CA_1,1914,0,HOBBIES_1_002_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,...,0,0,0,3,4,3.970703,0.137085,0.107117,0,-4.608542e+14
4,4,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,CA_1,1914,0,HOBBIES_1_003_CA_1_evaluation,d_1914,HOBBIES_1,HOBBIES,...,0,0,0,3,4,2.970703,0.379150,0.285645,0,-4.625018e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707435,189035,FOODS_3_825_WI_3_evaluation,FOODS_3_825,WI_3,1941,0,FOODS_3_825_WI_3_evaluation,d_1941,FOODS_3,FOODS,...,0,0,0,2,5,3.980469,1.377930,0.714355,2,-4.541656e+14
1707436,189036,FOODS_3_826_WI_3_validation,FOODS_3_826,WI_3,1941,0,FOODS_3_826_WI_3_evaluation,d_1941,FOODS_3,FOODS,...,0,0,0,2,5,1.280273,1.388672,1.952148,0,-4.576688e+14
1707437,189037,FOODS_3_826_WI_3_evaluation,FOODS_3_826,WI_3,1941,0,FOODS_3_826_WI_3_evaluation,d_1941,FOODS_3,FOODS,...,0,0,0,2,5,1.280273,1.388672,1.952148,0,-4.576688e+14
1707438,189038,FOODS_3_827_WI_3_validation,FOODS_3_827,WI_3,1941,0,FOODS_3_827_WI_3_evaluation,d_1941,FOODS_3,FOODS,...,0,0,0,2,5,1.000000,2.166016,1.809570,1,-4.570586e+14


#### Since the model isn't good at all, there is no need to save it for later use. However, if we wanted to use it later, we could pickle it as below.

In [18]:
import pickle
with open(export_path+'Model_2_sk_SGDRegressor_trained_model.pickle', 'wb') as f:
    pickle.dump(model, f)

## MODEL 3: SGD trained in batches

In [1]:
#####################################################################
#Export files to folder
export_path = 'data/outputs/'

#name of the final preprocessed file to be exported
final_preprocessed_file = 'sales_train_evaluation_preprocessed_final.csv'  #input('Enter the name of the final preprocessed file: ')

#name of the preprocessed validation file to be exported
validation_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_prediction.csv'

#name of the preprocessed evaluation_data to be exported
evaluation_file = 'submission_kaggle_preprocessed_to_be_used_4_private_LB_prediction.csv'

#name of the file to be exported and used for submission
submission_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_submission.csv'


#Store the datatypes prior to export to be able to use later when importing
schema={'id': 'O', 'item_id': 'O', 'dept_id': 'O', 'cat_id': 'O', 'store_id': 'O',
        'state_id': 'O', 'FOODS_2': 'int8', 'FOODS_3': 'int8', 'HOBBIES_1': 'int8',
        'HOBBIES_2': 'int8', 'HOUSEHOLD_1': 'int8', 'HOUSEHOLD_2': 'int8',
        'HOBBIES': 'int8', 'HOUSEHOLD': 'int8', 'TX': 'int8', 'WI': 'int8',
        'CA_2': 'int8', 'CA_3': 'int8', 'CA_4': 'int8', 'TX_1': 'int8', 'TX_2': 'int8',
        'TX_3': 'int8', 'WI_1': 'int8', 'WI_2': 'int8', 'WI_3': 'int8', 'd': 'O',
        'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int8', 'month': 'int8',
        'year': 'int16', 'snap_CA': 'int8', 'snap_TX': 'int8', 'snap_WI': 'int8',
        'day': 'int16', 'EidAlAdha': 'int8', 'OrthodoxEaster': 'int8', 'LaborDay': 'int8',
        'Chanukah End': 'int8', 'LentStart': 'int8', 'Halloween': 'int8', 'NBAFinalsEnd': 'int8',
        'Cinco De Mayo': 'int8', 'LentWeek2': 'int8', 'ColumbusDay': 'int8', 'Pesach End': 'int8',
        'OrthodoxChristmas': 'int8', 'Eid al-Fitr': 'int8', 'Easter': 'int8', "Father's day": 'int8',
        'NBAFinalsStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
        'SuperBowl': 'int8', 'Ramadan starts': 'int8', 'MemorialDay': 'int8', 'PresidentsDay': 'int8',
        'NewYear': 'int8', 'Thanksgiving': 'int8', 'Christmas': 'int8', 'Purim End': 'int8',
        "Mother's day": 'int8', 'StPatricksDay': 'int8', 'IndependenceDay': 'int8', 'VeteransDay': 'int8',
        'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'Sporting': 'int8',
        'sell_price': 'float16', 'weekly_avg_4weeks_ago': 'float16', 'last_month_avg': 'float16'}

#identify the columns necessary when importing preprocessed file
training_cols = organized_cols = [
         'id', 'day', 'item_id', 'store_id','wm_yr_wk', 'year',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago', 

         'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]


#save the submission file column names to a variable
submission_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']


#################################################################################
#SGD Batch training
import pdb

import pandas as pd
import numpy as np

import time
x=time.time()  #record the start time
##################################################################################

#Set parameters
epochs = 4
early_stop_threshold = 2
learning_rate = 0.000001
alpha = 0.1
verbose = True
batch_size = 1000000

weights={}


#initialize coefficients and the bias
np.random.seed(10)
W = np.random.rand(62, 1)
B = np.random.random(size=(1))
N = np.random.rand(batch_size,1)/5


#read in sales_train_validation_process 
training_df_chunks = pd.read_csv(export_path+final_preprocessed_file, usecols=training_cols, dtype=schema, chunksize=batch_size)
    
#initialize the loss and and a loss list to help with early stopping
losses = [0]
loss_score = 0
  

for epoch in range(epochs):
    
    
    loss = 0  #reset the loss for each epoch
                          
    for chunk in training_df_chunks:
        
        #pdb.set_trace()
                                 
        #drop missing values
        chunk.dropna(subset=['sell_price', 'last_month_avg', 'weekly_avg_4weeks_ago'], inplace=True)

        #filter chunk for days between d_1 and d_1913
        chunk = chunk.loc[chunk.day<1914]
        
        if chunk.shape[0] == 0:  #if chunk is null skip to the next loop
            continue

        X = chunk[input_cols]   #filter the dataset with the input cols

        target = np.array(chunk[target_col]).reshape(-1,1)   #labels


        #count the number of items
        num_observations = X.shape[0]

        N = np.random.rand(X.shape[0],1)/5  #random noise
                                 
        Y =  X.dot(W) +  B + N #calculate the targets
        
        Y[0] = Y[0].apply(lambda x: x if x>=0 else 0)  #negative estimates should be zero
        
        deltas = target - Y  #Find the error
                                 
        #scale the deltas
        deltas_scaled = deltas / num_observations

        #Optimize
        #update the weights biases
        w_momentum = alpha * learning_rate * np.dot(X.T, deltas_scaled)
        W = W - learning_rate * np.dot(X.T, deltas_scaled) - w_momentum

        b_momentum = alpha * learning_rate * np.sum(deltas_scaled)
        B = B - learning_rate * np.sum(deltas_scaled) - b_momentum
        
        #update the learning rate by 1%
        learning_rate = 0.99 * learning_rate
                                                
        #Calculate the loss                        
        loss += deltas_scaled.pow(2).sum()  #Loss function                      
                 
                                         
    #store the weights
    weights[epoch] = (W,B) #store the weights and biases 
        
    #set up an early stopping mechanism
    #add the loss to the losses list
    
    loss = pow(loss, 0.5) #loss as a RMSE function
    
    #Print loss and epoch
    if verbose == True:
        print(f' Epoch: {epoch}  Loss: {loss}')
            
    #store the losses that got larger
    if (losses[-1] < loss)[0] :
        loss_score += 1   #count the number of times loss got bigger
        losses.append(loss)  #keep track of the change in loss function
        
    else:
        loss_score = 0             
   
    if loss_score >= early_stop_threshold or epoch == (epochs-1):   #If the loss is increasing  or ran out of epochs stop learning
       
        break       
        
y= time.time()
print(f'It took {(y-x)/60} minutes to complete.')

#pickle the weights
import pickle
with open(export_path+'Model_3_Weights_SGD_Batchsize_1M_L.rate_0.000001.pickle', 'wb') as f:
    pickle.dump(weights[0], f)  #pickling the first weight[0]. We can also pickle all of them and test each one of them when making predictions

 Epoch: 0  Loss: 0    2.630248e+37
dtype: float64
 Epoch: 1  Loss: 0.0
 Epoch: 2  Loss: 0.0
 Epoch: 3  Loss: 0.0
It took 4.7606591065724695 minutes to complete.


#### It is clear that the algorithm couldn't learn with the current parameters and hyperparameters. Epoch, learning_rate, alpha value, and initializations must be adjusted through trial and error until the algorithm starts learning. I adjusted these settings to demonstrate how the algorithm works and to complete the run in a short time for practical reasons.

#### Making predictions

In [2]:
#####################################################################
#Export files to folder
export_path = 'data/outputs/'

#name of the final preprocessed file to be exported
final_preprocessed_file = 'sales_train_evaluation_preprocessed_final.csv'  #input('Enter the name of the final preprocessed file: ')

#name of the preprocessed validation file to be exported
validation_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_prediction.csv'

#name of the preprocessed evaluation_data to be exported
evaluation_file = 'submission_kaggle_preprocessed_to_be_used_4_private_LB_prediction.csv'

#name of the file to be exported and used for submission
submission_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_submission.csv'


#Store the datatypes prior to export to be able to use later when importing
schema={'id': 'O', 'item_id': 'O', 'dept_id': 'O', 'cat_id': 'O', 'store_id': 'O',
        'state_id': 'O', 'FOODS_2': 'int8', 'FOODS_3': 'int8', 'HOBBIES_1': 'int8',
        'HOBBIES_2': 'int8', 'HOUSEHOLD_1': 'int8', 'HOUSEHOLD_2': 'int8',
        'HOBBIES': 'int8', 'HOUSEHOLD': 'int8', 'TX': 'int8', 'WI': 'int8',
        'CA_2': 'int8', 'CA_3': 'int8', 'CA_4': 'int8', 'TX_1': 'int8', 'TX_2': 'int8',
        'TX_3': 'int8', 'WI_1': 'int8', 'WI_2': 'int8', 'WI_3': 'int8', 'd': 'O',
        'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int8', 'month': 'int8',
        'year': 'int16', 'snap_CA': 'int8', 'snap_TX': 'int8', 'snap_WI': 'int8',
        'day': 'int16', 'EidAlAdha': 'int8', 'OrthodoxEaster': 'int8', 'LaborDay': 'int8',
        'Chanukah End': 'int8', 'LentStart': 'int8', 'Halloween': 'int8', 'NBAFinalsEnd': 'int8',
        'Cinco De Mayo': 'int8', 'LentWeek2': 'int8', 'ColumbusDay': 'int8', 'Pesach End': 'int8',
        'OrthodoxChristmas': 'int8', 'Eid al-Fitr': 'int8', 'Easter': 'int8', "Father's day": 'int8',
        'NBAFinalsStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
        'SuperBowl': 'int8', 'Ramadan starts': 'int8', 'MemorialDay': 'int8', 'PresidentsDay': 'int8',
        'NewYear': 'int8', 'Thanksgiving': 'int8', 'Christmas': 'int8', 'Purim End': 'int8',
        "Mother's day": 'int8', 'StPatricksDay': 'int8', 'IndependenceDay': 'int8', 'VeteransDay': 'int8',
        'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'Sporting': 'int8',
        'sell_price': 'float16', 'weekly_avg_4weeks_ago': 'float16', 'last_month_avg': 'float16'}

#identify the columns necessary when importing preprocessed file
training_cols = organized_cols = [
         'id', 'day', 'item_id', 'store_id','wm_yr_wk', 'year',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago', 

         'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]


#save the submission file column names to a variable
submission_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']


#################################################################################
import pandas as pd
import numpy as np
import pickle
#################################################################################


with open(export_path+'Model_3_Weights_SGD_Batchsize_1M_L.rate_0.000001.pickle', 'rb') as f:
    weight_dict = pickle.load(f)
    
#read the validation file    
validation_df = pd.read_csv(export_path+validation_file, dtype=schema)

#choose the input columns from the validation file
x = validation_df[input_cols]
           
#dot product of the inputs with weights and additon of bias gives the prediction for each item
validation_df['predictions'] = x.dot(weight_dict[0]) + weight_dict[1]


kaggle_output_piv = validation_df.pivot_table(index='id', values='predictions', columns='day')
kaggle_output_piv.columns = submission_columns

#to export in submission format uncomment below
kaggle_output_piv.to_csv(export_path+'Model_3_submission_SGD_Batchsize_10K_L.rate_0.000001_result_to_be_used_4submission.csv')

#Review the output. Make predictions after the algortihm successfully learns and optimizes the weights.
#Otherwise, we expect to get very wrong results
kaggle_output_piv

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_evaluation,6.607168e+40,6.607174e+40,6.607181e+40,6.607187e+40,6.607194e+40,6.607154e+40,6.607168e+40,6.607174e+40,6.607181e+40,6.607187e+40,...,6.607200e+40,6.607163e+40,6.607170e+40,6.607175e+40,6.607182e+40,6.607188e+40,6.607195e+40,6.607201e+40,6.607161e+40,6.607167e+40
FOODS_1_001_CA_1_validation,6.607168e+40,6.607174e+40,6.607181e+40,6.607187e+40,6.607194e+40,6.607154e+40,6.607168e+40,6.607174e+40,6.607181e+40,6.607187e+40,...,6.607200e+40,6.607163e+40,6.607170e+40,6.607175e+40,6.607182e+40,6.607188e+40,6.607195e+40,6.607201e+40,6.607161e+40,6.607167e+40
FOODS_1_001_CA_2_evaluation,6.607168e+40,6.607174e+40,6.607181e+40,6.607187e+40,6.607194e+40,6.607154e+40,6.607168e+40,6.607174e+40,6.607181e+40,6.607187e+40,...,6.607200e+40,6.607163e+40,6.607170e+40,6.607176e+40,6.607182e+40,6.607189e+40,6.607195e+40,6.607201e+40,6.607161e+40,6.607168e+40
FOODS_1_001_CA_2_validation,6.607168e+40,6.607174e+40,6.607181e+40,6.607187e+40,6.607194e+40,6.607154e+40,6.607168e+40,6.607174e+40,6.607181e+40,6.607187e+40,...,6.607200e+40,6.607163e+40,6.607170e+40,6.607176e+40,6.607182e+40,6.607189e+40,6.607195e+40,6.607201e+40,6.607161e+40,6.607168e+40
FOODS_1_001_CA_3_evaluation,6.607168e+40,6.607174e+40,6.607181e+40,6.607187e+40,6.607194e+40,6.607154e+40,6.607168e+40,6.607174e+40,6.607181e+40,6.607187e+40,...,6.607200e+40,6.607163e+40,6.607170e+40,6.607176e+40,6.607182e+40,6.607189e+40,6.607195e+40,6.607201e+40,6.607161e+40,6.607168e+40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_WI_1_validation,6.607192e+40,6.607199e+40,6.607205e+40,6.607212e+40,6.607218e+40,6.607180e+40,6.607193e+40,6.607199e+40,6.607206e+40,6.607212e+40,...,6.607225e+40,6.607186e+40,6.607193e+40,6.607199e+40,6.607205e+40,6.607211e+40,6.607218e+40,6.607224e+40,6.607186e+40,6.607192e+40
HOUSEHOLD_2_516_WI_2_evaluation,6.607192e+40,6.607199e+40,6.607205e+40,6.607212e+40,6.607218e+40,6.607180e+40,6.607193e+40,6.607199e+40,6.607206e+40,6.607212e+40,...,6.607225e+40,6.607186e+40,6.607193e+40,6.607199e+40,6.607205e+40,6.607211e+40,6.607218e+40,6.607224e+40,6.607186e+40,6.607192e+40
HOUSEHOLD_2_516_WI_2_validation,6.607192e+40,6.607199e+40,6.607205e+40,6.607212e+40,6.607218e+40,6.607180e+40,6.607193e+40,6.607199e+40,6.607206e+40,6.607212e+40,...,6.607225e+40,6.607186e+40,6.607193e+40,6.607199e+40,6.607205e+40,6.607211e+40,6.607218e+40,6.607224e+40,6.607186e+40,6.607192e+40


Algorithm worked but results are extremely poor as predicted. We need more than 4 epochs and more sophisticated weight/bias initialization. Also, a variable learning rate schedule could improve the result. However, multiple linear regression isn't a suitable for this problem even with ML. I used this model only for teaching purposes since the logic behind it applies in many situations.

## MODEL 4: RandomForest Regressor

In [1]:
#############################################
#Export files to folder
export_path = 'data/outputs/'

#name of the final preprocessed file to be exported
final_preprocessed_file = 'sales_train_evaluation_preprocessed_final.csv'  #input('Enter the name of the final preprocessed file: ')

#name of the preprocessed validation file to be exported
validation_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_prediction.csv'

#name of the preprocessed evaluation_data to be exported
evaluation_file = 'submission_kaggle_preprocessed_to_be_used_4_private_LB_prediction.csv'

#name of the file to be exported and used for submission
submission_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_submission.csv'


#Store the datatypes prior to export to be able to use later when importing
schema={'id': 'O', 'item_id': 'O', 'dept_id': 'O', 'cat_id': 'O', 'store_id': 'O',
        'state_id': 'O', 'FOODS_2': 'int8', 'FOODS_3': 'int8', 'HOBBIES_1': 'int8',
        'HOBBIES_2': 'int8', 'HOUSEHOLD_1': 'int8', 'HOUSEHOLD_2': 'int8',
        'HOBBIES': 'int8', 'HOUSEHOLD': 'int8', 'TX': 'int8', 'WI': 'int8',
        'CA_2': 'int8', 'CA_3': 'int8', 'CA_4': 'int8', 'TX_1': 'int8', 'TX_2': 'int8',
        'TX_3': 'int8', 'WI_1': 'int8', 'WI_2': 'int8', 'WI_3': 'int8', 'd': 'O',
        'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int8', 'month': 'int8',
        'year': 'int16', 'snap_CA': 'int8', 'snap_TX': 'int8', 'snap_WI': 'int8',
        'day': 'int16', 'EidAlAdha': 'int8', 'OrthodoxEaster': 'int8', 'LaborDay': 'int8',
        'Chanukah End': 'int8', 'LentStart': 'int8', 'Halloween': 'int8', 'NBAFinalsEnd': 'int8',
        'Cinco De Mayo': 'int8', 'LentWeek2': 'int8', 'ColumbusDay': 'int8', 'Pesach End': 'int8',
        'OrthodoxChristmas': 'int8', 'Eid al-Fitr': 'int8', 'Easter': 'int8', "Father's day": 'int8',
        'NBAFinalsStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
        'SuperBowl': 'int8', 'Ramadan starts': 'int8', 'MemorialDay': 'int8', 'PresidentsDay': 'int8',
        'NewYear': 'int8', 'Thanksgiving': 'int8', 'Christmas': 'int8', 'Purim End': 'int8',
        "Mother's day": 'int8', 'StPatricksDay': 'int8', 'IndependenceDay': 'int8', 'VeteransDay': 'int8',
        'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'Sporting': 'int8',
        'sell_price': 'float16', 'weekly_avg_4weeks_ago': 'float16', 'last_month_avg': 'float16'}

#identify the columns necessary when importing preprocessed file
training_cols = organized_cols = [
         'id', 'day', 'item_id', 'store_id','wm_yr_wk', 'year',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago', 

         'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[5:-1]
target_col = training_cols[-1]

#save the submission file column names to a variable
submission_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']


#################################################################################

import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor

#import the training data
training_df = pd.read_csv(export_path+final_preprocessed_file, dtype=schema, usecols=training_cols)

#drop the rows with missing values
training_df = training_df.dropna(subset=['sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago'])

#read the validation file. 
validation_df = pd.read_csv(export_path+validation_file, usecols=training_cols, dtype=schema)

#save the results to a dataframe
submission = pd.DataFrame()
validation =pd.DataFrame()

#Create a RandomForest instance
model = RandomForestRegressor(random_state=1 , n_jobs=4, random_state=1)

#save the unique items in a list
items = training_df.item_id.unique()

In [3]:
#read the validation file. 
validation_df = pd.read_csv(export_path+validation_file, usecols=training_cols, dtype=schema)
i=0
for item in items:   #iterate through the items
    print(item)
    i+=1
    x = training_df[training_df['item_id'] == item][input_cols]   #filter the dataset on the item_id
    y = training_df[training_df['item_id'] == item].quantity_sold
    
    model.fit(x, y)   #train the model

    #Make predictions
    #Prepare a submission file with the validation file
    #create a new result dataframe to save predictions
    result_v = validation_df[validation_df['item_id'] == item][['id', 'item_id', 'day', 'quantity_sold']]   

    #save the predictions
    result_v['predictions_val'] = model.predict(validation_df[validation_df['item_id'] == item][input_cols])  
    validation = pd.concat([validation,result_v])


HOBBIES_1_008
HOBBIES_1_009
HOBBIES_1_010
HOBBIES_1_012
HOBBIES_1_015
HOBBIES_1_016
HOBBIES_1_020
HOBBIES_1_022
HOBBIES_1_023
HOBBIES_1_028
HOBBIES_1_029
HOBBIES_1_031
HOBBIES_1_032
HOBBIES_1_034
HOBBIES_1_036
HOBBIES_1_038
HOBBIES_1_043
HOBBIES_1_044
HOBBIES_1_047
HOBBIES_1_050
HOBBIES_1_055
HOBBIES_1_058
HOBBIES_1_061
HOBBIES_1_063
HOBBIES_1_064
HOBBIES_1_067
HOBBIES_1_068
HOBBIES_1_075
HOBBIES_1_076
HOBBIES_1_078
HOBBIES_1_079
HOBBIES_1_080
HOBBIES_1_082
HOBBIES_1_084
HOBBIES_1_086
HOBBIES_1_088
HOBBIES_1_089
HOBBIES_1_092
HOBBIES_1_097
HOBBIES_1_098
HOBBIES_1_099
HOBBIES_1_100
HOBBIES_1_103
HOBBIES_1_105
HOBBIES_1_106
HOBBIES_1_108
HOBBIES_1_110
HOBBIES_1_111
HOBBIES_1_112
HOBBIES_1_117
HOBBIES_1_118
HOBBIES_1_127
HOBBIES_1_128
HOBBIES_1_130
HOBBIES_1_132
HOBBIES_1_137
HOBBIES_1_141
HOBBIES_1_143
HOBBIES_1_144
HOBBIES_1_145
HOBBIES_1_146
HOBBIES_1_150
HOBBIES_1_151
HOBBIES_1_157
HOBBIES_1_159
HOBBIES_1_163
HOBBIES_1_164
HOBBIES_1_169
HOBBIES_1_172
HOBBIES_1_175
HOBBIES_1_178
HOBBIE

HOUSEHOLD_2_215
HOUSEHOLD_2_217
HOUSEHOLD_2_218
HOUSEHOLD_2_219
HOUSEHOLD_2_220
HOUSEHOLD_2_222
HOUSEHOLD_2_223
HOUSEHOLD_2_224
HOUSEHOLD_2_226
HOUSEHOLD_2_227
HOUSEHOLD_2_228
HOUSEHOLD_2_231
HOUSEHOLD_2_233
HOUSEHOLD_2_234
HOUSEHOLD_2_235
HOUSEHOLD_2_236
HOUSEHOLD_2_238
HOUSEHOLD_2_239
HOUSEHOLD_2_240
HOUSEHOLD_2_241
HOUSEHOLD_2_242
HOUSEHOLD_2_243
HOUSEHOLD_2_244
HOUSEHOLD_2_246
HOUSEHOLD_2_247
HOUSEHOLD_2_249
HOUSEHOLD_2_250
HOUSEHOLD_2_253
HOUSEHOLD_2_254
HOUSEHOLD_2_255
HOUSEHOLD_2_256
HOUSEHOLD_2_259
HOUSEHOLD_2_260
HOUSEHOLD_2_261
HOUSEHOLD_2_262
HOUSEHOLD_2_268
HOUSEHOLD_2_269
HOUSEHOLD_2_270
HOUSEHOLD_2_271
HOUSEHOLD_2_273
HOUSEHOLD_2_274
HOUSEHOLD_2_275
HOUSEHOLD_2_278
HOUSEHOLD_2_281
HOUSEHOLD_2_282
HOUSEHOLD_2_283
HOUSEHOLD_2_284
HOUSEHOLD_2_285
HOUSEHOLD_2_287
HOUSEHOLD_2_288
HOUSEHOLD_2_291
HOUSEHOLD_2_292
HOUSEHOLD_2_293
HOUSEHOLD_2_295
HOUSEHOLD_2_297
HOUSEHOLD_2_299
HOUSEHOLD_2_301
HOUSEHOLD_2_304
HOUSEHOLD_2_305
HOUSEHOLD_2_306
HOUSEHOLD_2_308
HOUSEHOLD_2_310
HOUSEHOL

FOODS_3_305
FOODS_3_311
FOODS_3_314
FOODS_3_317
FOODS_3_318
FOODS_3_319
FOODS_3_324
FOODS_3_326
FOODS_3_328
FOODS_3_330
FOODS_3_331
FOODS_3_334
FOODS_3_336
FOODS_3_339
FOODS_3_341
FOODS_3_342
FOODS_3_343
FOODS_3_345
FOODS_3_346
FOODS_3_347
FOODS_3_348
FOODS_3_349
FOODS_3_351
FOODS_3_352
FOODS_3_355
FOODS_3_356
FOODS_3_357
FOODS_3_363
FOODS_3_367
FOODS_3_368
FOODS_3_369
FOODS_3_373
FOODS_3_374
FOODS_3_375
FOODS_3_376
FOODS_3_377
FOODS_3_378
FOODS_3_380
FOODS_3_383
FOODS_3_387
FOODS_3_388
FOODS_3_389
FOODS_3_392
FOODS_3_394
FOODS_3_395
FOODS_3_397
FOODS_3_398
FOODS_3_400
FOODS_3_401
FOODS_3_404
FOODS_3_405
FOODS_3_406
FOODS_3_407
FOODS_3_408
FOODS_3_410
FOODS_3_411
FOODS_3_412
FOODS_3_420
FOODS_3_422
FOODS_3_424
FOODS_3_432
FOODS_3_435
FOODS_3_446
FOODS_3_448
FOODS_3_449
FOODS_3_450
FOODS_3_452
FOODS_3_458
FOODS_3_460
FOODS_3_461
FOODS_3_462
FOODS_3_464
FOODS_3_465
FOODS_3_467
FOODS_3_468
FOODS_3_469
FOODS_3_470
FOODS_3_473
FOODS_3_475
FOODS_3_476
FOODS_3_477
FOODS_3_478
FOODS_3_480
FOOD

HOUSEHOLD_2_192
HOUSEHOLD_2_504
FOODS_3_809
FOODS_2_336
FOODS_3_179
FOODS_3_251
FOODS_3_104
HOBBIES_1_198
FOODS_1_008
HOBBIES_2_052
FOODS_1_048
FOODS_1_050
FOODS_1_158
FOODS_3_140
HOUSEHOLD_1_406
FOODS_3_181
HOUSEHOLD_1_155
FOODS_3_669
HOUSEHOLD_2_039
HOUSEHOLD_2_149
FOODS_1_147
FOODS_3_142
HOUSEHOLD_1_305
FOODS_3_268
HOBBIES_1_142
FOODS_3_570
FOODS_3_801
FOODS_3_258
FOODS_3_259
HOBBIES_1_069
HOUSEHOLD_2_405
FOODS_3_130
FOODS_3_321
FOODS_1_105
FOODS_3_402
FOODS_1_184
HOUSEHOLD_2_121
HOUSEHOLD_2_436
HOUSEHOLD_1_032
HOBBIES_1_153
HOUSEHOLD_2_053
FOODS_3_315
HOUSEHOLD_2_458
FOODS_3_625
FOODS_1_043
FOODS_1_058
FOODS_1_110
FOODS_1_120
FOODS_2_084
HOUSEHOLD_2_004
FOODS_3_149
FOODS_3_457
FOODS_3_428
HOUSEHOLD_2_265
FOODS_3_675
HOBBIES_1_281
FOODS_3_138
FOODS_3_454
FOODS_3_758
FOODS_3_816
HOUSEHOLD_2_086
FOODS_3_320
FOODS_2_169
FOODS_2_309
HOUSEHOLD_2_123
FOODS_3_566
FOODS_3_573
HOUSEHOLD_1_064
HOUSEHOLD_2_230
HOUSEHOLD_1_158
HOUSEHOLD_2_320
HOUSEHOLD_2_484
FOODS_2_234
FOODS_2_273
FOODS_3_185


FOODS_2_151
FOODS_3_299
HOUSEHOLD_2_046
HOBBIES_1_218
FOODS_1_210
FOODS_2_383
HOBBIES_1_121
HOUSEHOLD_1_317
FOODS_3_579
HOBBIES_1_056
HOUSEHOLD_2_196
FOODS_3_589
HOBBIES_2_139
FOODS_3_689
HOUSEHOLD_1_029
FOODS_3_008
HOUSEHOLD_1_257
HOUSEHOLD_1_431
HOUSEHOLD_1_500
FOODS_1_089
HOBBIES_1_140
HOBBIES_1_223
HOUSEHOLD_1_419
FOODS_3_158
FOODS_3_253
FOODS_3_361
FOODS_3_430
FOODS_3_074
FOODS_3_233
FOODS_3_754
HOUSEHOLD_1_217
HOUSEHOLD_1_220
HOUSEHOLD_2_397
FOODS_1_082
FOODS_2_078
HOUSEHOLD_2_061
HOUSEHOLD_2_343
FOODS_3_106
FOODS_3_147
FOODS_3_519
FOODS_3_803
HOUSEHOLD_1_020
HOUSEHOLD_2_513
FOODS_2_108
FOODS_1_099
FOODS_3_600
FOODS_3_821
HOBBIES_2_027
HOUSEHOLD_2_203
HOUSEHOLD_2_266
FOODS_3_071
HOBBIES_1_148
HOBBIES_2_144
HOUSEHOLD_1_320
HOUSEHOLD_2_303
HOUSEHOLD_2_496
FOODS_3_826
HOBBIES_2_114
HOUSEHOLD_1_363
HOUSEHOLD_2_319
HOBBIES_1_053
HOBBIES_2_081
HOBBIES_2_094
FOODS_2_227
HOBBIES_2_015
HOBBIES_1_051
HOUSEHOLD_1_202
HOUSEHOLD_1_267
FOODS_2_179
FOODS_3_075
FOODS_2_125
HOUSEHOLD_1_152
FOODS_

HOBBIES_2_023
HOUSEHOLD_1_388
HOUSEHOLD_1_253
HOUSEHOLD_1_209
HOUSEHOLD_1_390
FOODS_2_102
FOODS_2_250
FOODS_2_271
FOODS_2_147
FOODS_2_017
HOUSEHOLD_1_156
HOUSEHOLD_1_227
HOUSEHOLD_2_066
FOODS_3_796
FOODS_1_131
HOBBIES_1_204
FOODS_1_122
HOUSEHOLD_2_216
HOUSEHOLD_1_208
FOODS_1_179
FOODS_2_252
FOODS_2_097
FOODS_3_743
HOBBIES_1_046
FOODS_3_760
FOODS_3_255
FOODS_2_215
FOODS_3_260
HOUSEHOLD_1_137
FOODS_1_160
FOODS_3_517
HOBBIES_1_269
HOUSEHOLD_1_274
HOBBIES_1_125
HOUSEHOLD_1_246
HOUSEHOLD_1_476
HOUSEHOLD_2_057
HOUSEHOLD_2_418
HOUSEHOLD_1_201
HOUSEHOLD_1_484
HOUSEHOLD_1_489
HOUSEHOLD_1_534
FOODS_3_613
FOODS_2_185
HOBBIES_1_232
HOBBIES_1_344
FOODS_3_254
FOODS_3_366
FOODS_3_472
HOUSEHOLD_1_308
FOODS_2_390
HOUSEHOLD_1_526
HOUSEHOLD_2_186
FOODS_3_003
FOODS_3_353
FOODS_3_278
FOODS_3_466
HOUSEHOLD_1_242
FOODS_3_500
FOODS_3_563
FOODS_3_047
FOODS_3_166
FOODS_3_647
FOODS_3_038
FOODS_3_119
HOUSEHOLD_1_159


In [4]:
#Submission table has ['id', 'item_id', 'day', 'quantity_sold'] columns. To prepare it for the submission,
#we need to drop unneeded columns, create a pivot table, and rename the columns.

kaggle_output = validation.drop(['item_id','quantity_sold'], axis=1)
kaggle_output_piv = kaggle_output.pivot_table(index=['id'], columns='day', values='predictions_val')
kaggle_output_piv.columns = submission_columns

#to export in submission format uncomment below
kaggle_output_piv.to_csv(export_path+'Model_4_submission_sklearn_RandomForest_result_to_be_used_4submission.csv')
kaggle_output_piv

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_evaluation,1.88,0.32,0.23,0.10,0.57,1.115,1.01,0.77,3.71,0.96,...,1.490,2.08,1.26,0.29,0.26,0.14,0.63,0.24,0.27,0.19
FOODS_1_001_CA_1_validation,1.88,0.32,0.23,0.10,0.57,1.115,1.01,0.77,3.71,0.96,...,1.490,2.08,1.26,0.29,0.26,0.14,0.63,0.24,0.27,0.19
FOODS_1_001_CA_2_evaluation,0.73,1.98,0.38,0.35,0.13,0.950,2.94,0.33,0.54,0.48,...,0.940,0.33,0.64,0.81,0.99,0.26,0.22,0.89,1.46,0.78
FOODS_1_001_CA_2_validation,0.73,1.98,0.38,0.35,0.13,0.950,2.94,0.33,0.54,0.48,...,0.940,0.33,0.64,0.81,0.99,0.26,0.22,0.89,1.46,0.78
FOODS_1_001_CA_3_evaluation,0.93,0.26,0.71,0.22,5.23,1.610,0.31,0.07,0.97,0.16,...,1.030,1.78,1.58,0.28,0.16,0.64,0.27,2.30,1.78,1.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_516_WI_1_validation,0.67,0.13,0.09,0.62,0.18,0.040,0.07,0.05,0.00,0.01,...,0.210,0.19,0.03,0.00,0.02,0.03,0.01,0.75,0.07,0.00
HOUSEHOLD_2_516_WI_2_evaluation,0.03,0.06,0.05,0.09,0.02,0.000,0.02,0.04,0.00,0.04,...,0.020,0.14,0.59,0.63,0.13,0.00,0.06,0.01,0.03,0.07
HOUSEHOLD_2_516_WI_2_validation,0.03,0.06,0.05,0.09,0.02,0.000,0.02,0.04,0.00,0.04,...,0.020,0.14,0.59,0.63,0.13,0.00,0.06,0.01,0.03,0.07


#### Now, let's get the results on Kaggle Leaderboard. We should get a much better result compared to linear regression models we tried above. 

Your Best Entry 
You advanced 3,802 places on the leaderboard!

Your submission scored 0.25442, which is an improvement of your previous score of 0.90096. Great job!

#### Feature Engineering helped with the results significantly. Without the two enginered features, RandomForestRegressor RMSE was 0.901. With the added feature that reflected the weekly and monthly trends, RMSE dropped to 0.25

## MODEL 5: RFR with avg weekly WMT stock prices from 4 weeks ago

#### More Feature Engineering to the stock prices

In [ ]:
import pandas as pd
pd.set_option('max_rows', 999)
pd.set_option('max_columns', None)

import numpy as np

import pandas_datareader.data as web
import datetime

import time

In [ ]:
#import sample_submission.csv from
sample_submission_path = 'data/sample_submission.csv'

#import sell_price.csv from
sell_prices_path = 'data/sell_prices.csv'

#import sales_train_validation.csv from
#sales_train_validation_path = 'data/sales_train_validation.csv'

#import sales_train_evaluation.csv from
sales_train_validation_path = 'data/sales_train_evaluation.csv'

#import calendar.csv from
calendar_path = 'data/calendar.csv'

#############################################
#Export files to folder
export_path = 'data/19_outputs/'

#name of the final preprocessed file to be exported
final_preprocessed_file = 'sales_train_evaluation_preprocessed_final.csv'  #input('Enter the name of the final preprocessed file: ')

#name of the preprocessed validation file to be exported
validation_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_prediction.csv'

#name of the preprocessed evaluation_data to be exported
evaluation_file = 'submission_kaggle_preprocessed_to_be_used_4_private_LB_prediction.csv'

#name of the file to be exported and used for submission
submission_file = 'submission_kaggle_preprocessed_to_be_used_4_public_LB_submission.csv'
#############################################


#Store the datatypes prior to export to be able to use later when importing
schema={'id': 'O', 'item_id': 'O', 'dept_id': 'O', 'cat_id': 'O', 'store_id': 'O',
        'state_id': 'O', 'FOODS_2': 'int8', 'FOODS_3': 'int8', 'HOBBIES_1': 'int8',
        'HOBBIES_2': 'int8', 'HOUSEHOLD_1': 'int8', 'HOUSEHOLD_2': 'int8',
        'HOBBIES': 'int8', 'HOUSEHOLD': 'int8', 'TX': 'int8', 'WI': 'int8',
        'CA_2': 'int8', 'CA_3': 'int8', 'CA_4': 'int8', 'TX_1': 'int8', 'TX_2': 'int8',
        'TX_3': 'int8', 'WI_1': 'int8', 'WI_2': 'int8', 'WI_3': 'int8', 'd': 'O',
        'quantity_sold': 'int16', 'wm_yr_wk': 'int16', 'wday': 'int8', 'month': 'int8',
        'year': 'int16', 'snap_CA': 'int8', 'snap_TX': 'int8', 'snap_WI': 'int8',
        'day': 'int16', 'EidAlAdha': 'int8', 'OrthodoxEaster': 'int8', 'LaborDay': 'int8',
        'Chanukah End': 'int8', 'LentStart': 'int8', 'Halloween': 'int8', 'NBAFinalsEnd': 'int8',
        'Cinco De Mayo': 'int8', 'LentWeek2': 'int8', 'ColumbusDay': 'int8', 'Pesach End': 'int8',
        'OrthodoxChristmas': 'int8', 'Eid al-Fitr': 'int8', 'Easter': 'int8', "Father's day": 'int8',
        'NBAFinalsStart': 'int8', 'MartinLutherKingDay': 'int8', 'ValentinesDay': 'int8',
        'SuperBowl': 'int8', 'Ramadan starts': 'int8', 'MemorialDay': 'int8', 'PresidentsDay': 'int8',
        'NewYear': 'int8', 'Thanksgiving': 'int8', 'Christmas': 'int8', 'Purim End': 'int8',
        "Mother's day": 'int8', 'StPatricksDay': 'int8', 'IndependenceDay': 'int8', 'VeteransDay': 'int8',
        'Cultural': 'int8', 'National': 'int8', 'Religious': 'int8', 'Sporting': 'int8',
        'sell_price': 'float16', 'weekly_avg_4weeks_ago': 'float16', 'last_month_avg': 'float16', 'last_28d_W_stock_avg':'float16'}

#identify the columns necessary when importing preprocessed file
training_cols = [
         'id','item_id', 'wm_yr_wk','day',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago',

         'quantity_sold']


#save the submission file column names to a variable
submission_columns = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']


#################################################################################

In [ ]:
#importing the calendar_df.csv
calendar_df = pd.read_csv(calendar_path)

#parse the date columns
calendar_df['date'] = pd.to_datetime(calendar_df['date'])

#save the first and the last dates
start=calendar_df.date.iloc[0].date()
end=calendar_df.date.iloc[-1].date()

#save the historical walmart stock prices as an indicator of economy
walmart_stock = web.get_data_yahoo("WMT", start, end)

#Merge the stock prices with the calendar_df
calendar_df.set_index('date', inplace=True)
calendar_df_walmart = calendar_df.join(walmart_stock['Close'])

#Save the Walmart's closing stock price
calendar_df = calendar_df_walmart.merge(calendar_df_walmart[['wm_yr_wk','Close']].groupby('wm_yr_wk').mean(), on='wm_yr_wk', how='left', suffixes=['','_WMT_stock'])
calendar_df.rename(columns={'Close_WMT_stock':'last_28d_W_stock_avg'}, inplace=True )

#add a day column
calendar_df['day'] = calendar_df.d.apply(lambda x: int(x.split('_')[1]))

#Drop the close column
calendar_df.drop('Close', axis=1, inplace=True)

#shift the days 28 days
calendar = calendar_df.copy()
calendar['day'] = calendar['day']+28

#Merging stock prices with final preprocessed data
final_preprocessed_data = pd.read_csv(export_path+final_preprocessed_file, dtype=schema,usecols=training_cols, chunksize=1000000, index_col=0)
writeheader=True
for chunk in final_preprocessed_data:
    chunk.merge(calendar[['day','last_28d_W_stock_avg']], on=['day'], how='left').to_csv(export_path+'with_stockprice_'+final_preprocessed_file,header=writeheader,mode='a')
    writeheader=False

#rename the final preprocessed file
final_preprocessed_file='with_stockprice_'+final_preprocessed_file

In [ ]:
#add  'last_28d_W_stock_avg' to training columns
#identify the columns necessary when importing preprocessed file
training_cols = [
         'id','item_id', 'wm_yr_wk','day',

         'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'HOBBIES', 'HOUSEHOLD', 

         'TX', 'WI', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3','WI_1', 'WI_2', 'WI_3',

         'snap_CA', 'snap_TX', 'snap_WI',

         'Cinco De Mayo', 'OrthodoxChristmas', "Father's day", 'Thanksgiving', "Mother's day", 'NBAFinalsStart', 
         'Christmas', 'Chanukah End', 'EidAlAdha', 'MemorialDay', 'Halloween', 'VeteransDay', 'SuperBowl', 'LentStart', 
         'MartinLutherKingDay', 'ValentinesDay', 'LaborDay', 'Easter', 'NBAFinalsEnd', 'Purim End', 'IndependenceDay',
         'Ramadan starts', 'StPatricksDay', 'LentWeek2', 'OrthodoxEaster', 'Pesach End', 'NewYear', 'PresidentsDay', 
         'Eid al-Fitr', 'ColumbusDay', 'Sporting', 'Cultural', 'National', 'Religious', 

         'wday', 'month','sell_price', 'last_month_avg',  'weekly_avg_4weeks_ago','last_28d_W_stock_avg',

         'quantity_sold']

#Save the input and target columns into variables
input_cols = training_cols[4:-1]
target_col = training_cols[-1]

In [ ]:
#and read the validation_df data
validation_df = pd.read_csv(export_path+validation_file, dtype=schema, index_col=0)

#merge the previous weekly avg stock prices from 4 weeks ago with the validation data
validation_df = validation_df.merge(calendar[['day', 'last_28d_W_stock_avg']], on=['day'], how='left')
#Reorganize the columns
validation_df = validation_df[training_cols]

#read in sales_train_validation_process 
training_df = pd.read_csv(export_path+final_preprocessed_file, dtype=schema,index_col=0)

#save the days1-1913 for training
training_df = training_df.loc[training_df.day<1914]

#Drop the rows with missing values
training_df.dropna(subset=['sell_price','last_month_avg','weekly_avg_4weeks_ago','quantity_sold','last_28d_W_stock_avg'], inplace=True)



### Build the model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#save the days1-1913 for training
training_df = training_df.loc[training_df.day<1914]

#save the results to a dataframe
submission = pd.DataFrame()
validation =pd.DataFrame()

#Create a RandomForest instance
model = RandomForestRegressor(random_state=1, verbose=1, n_jobs=-1)

#save the unique items in a list
items = validation_df.item_id.unique()
writeheader=True
i=1

for item in items:   #iterate through the items
    
    print(f'{i}. {item})
   
    x = training_df[training_df['item_id'] == item][input_cols]   #filter the dataset on the item_id
    y = training_df[training_df['item_id'] == item].quantity_sold
    
    model.fit(x, y)   #train the model

    #Make predictions
    #Prepare a submission file with the validation file
    #create a new result dataframe to save predictions
    result_v = validation_df[validation_df['item_id'] == item][['id', 'item_id', 'day', 'quantity_sold']]   

    #save the predictions
    result_v['predictions_val'] = model.predict(validation_df[validation_df['item_id'] == item][input_cols])
    result_v.to_csv(export_path+'model5_validation_predictions_raw.csv', mode='a', header=writeheader)
    writeheader=False
    validation = pd.concat([validation,result_v])

In [ ]:
#Submission table has ['id', 'item_id', 'day', 'quantity_sold'] columns. To prepare it for the submission,
#we need to drop unneeded columns, create a pivot table, and rename the columns.

kaggle_output = validation.drop(['item_id','quantity_sold'], axis=1)
kaggle_output_piv = kaggle_output.pivot_table(index=['id'], columns='day', values='predictions_val')
kaggle_output_piv.columns = submission_columns

#to export in submission format uncomment below
kaggle_output_piv.to_csv(export_path+'Model_5_submission_sklearn_RandomForest_result_to_be_used_4submission.csv')
kaggle_output_piv